Nombres y apellidos: Fiorella Meza Rodriguez <br>
Código: 20192730G

# Parte 1

## Ejercicio 0.1.1

Dadas tres oraciones "all models are wrong", a model is wrong, y some models are useful y un vocabulario ["<s>", "</s>", 'a', 'all', 'are', 'model', 'models',
               'some', 'useful', 'wrong']. En codigo responda las siguientes preguntas.

### a.Calcule las probabilidades de todos los bigramas sin suavizado

In [1]:
from collections import Counter
from collections import defaultdict

corpus = [ "all models are wrong", "a model is wrong",
            "some models are useful"]

vocabulario = ['<s>', '</s>', 'a', 'all', 'are', 'model', 'models',
               'some', 'useful', 'wrong']

In [3]:
# Inicializamos los conteos
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

# Poblamos los conteos
for sentence in corpus:
    words = sentence.split()
    for i in range(len(words) - 1):
        bigram_counts[(words[i], words[i+1])] += 1
        unigram_counts[words[i]] += 1
    unigram_counts[words[-1]] += 1


## Probabilidad de los bigramas
def bigram_probability(w1, w2):
    return bigram_counts[(w1, w2)] / unigram_counts[w1] if unigram_counts[w1] > 0 else 0


print("Probabilidades de bigramas sin suavizado:")
for i in bigram_counts:
  prob = bigram_probability(i[0],i[1])
  print(f"P('{i[1]}' | '{i[0]}') = {prob:.3f}")

Probabilidades de bigramas sin suavizado:
P('models' | 'all') = 1.000
P('are' | 'models') = 1.000
P('wrong' | 'are') = 0.500
P('model' | 'a') = 1.000
P('is' | 'model') = 1.000
P('wrong' | 'is') = 1.000
P('models' | 'some') = 1.000
P('useful' | 'are') = 0.500


### b. Calcule todas las probabilidades de todos los bigramas y el brigrama no visto 'a models' con suavizado add-one

Referencia: https://github.com/kapumota/Actividades-CC0C2/blob/3d0cde28776954bfc6367f2b94725dffcfaf8f38/Cuadernos-CC0C2/Clase2/Counts-backoff-suavizado.ipynb#L113

In [4]:
from collections import defaultdict

bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

corpus = ["all models are wrong", "a model is wrong",
            "some models are useful"]

# Poblamos los conteos
for sentence in corpus:
    words = sentence.split()
    for i in range(len(words) - 1):
        bigram_counts[(words[i], words[i+1])] += 1
        unigram_counts[words[i]] += 1
    unigram_counts[words[-1]] += 1


beta = 1
vocab_size = len(vocabulario)

def laplace_smoothing_bigram_probability(w1, w2, beta, vocab_size):
    bigram_prob = (bigram_counts[(w1, w2)] + beta) / (unigram_counts[w1] + beta * vocab_size)
    return bigram_prob

Ahora calcularemos de todos los bigramas:

In [6]:
print(f"La probabilidad de los bigramas con suavizado de laplace:")
for bigram in bigram_counts:
  prob = laplace_smoothing_bigram_probability(bigram[0],bigram[1], beta, vocab_size)
  print(f"P('{bigram[1]}' | '{bigram[0]}') = {prob:.4f}")

La probabilidad de los bigramas con suavizado de laplace:
P('models' | 'all') = 0.1818
P('are' | 'models') = 0.2500
P('wrong' | 'are') = 0.1667
P('model' | 'a') = 0.1818
P('is' | 'model') = 0.1818
P('wrong' | 'is') = 0.1818
P('models' | 'some') = 0.1818
P('useful' | 'are') = 0.1667


In [7]:
prob = laplace_smoothing_bigram_probability("a", "models", beta, vocab_size)
print(f"La probabilidad del bigrama 'a models' con suavizado de Laplace es:")
print(f"P('models' | 'a') = {prob:.4f}")

La probabilidad del bigrama 'a models' con suavizado de Laplace es:
P('models' | 'a') = 0.0909


### C. Calcule la probabilidad de todos los bigramas y el bigrama no visto "a models" con suavizado add-k. Pruebe con k=00.5 y k=0.15

Referencia: https://github.com/kapumota/Actividades-CC0C2/blob/3d0cde28776954bfc6367f2b94725dffcfaf8f38/Cuadernos-CC0C2/Clase2/Modelos-lenguaje2.ipynb#L12

In [8]:
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

#Suavizado add-k para bigrama
def add_k_smoothing_bigram(corpus, k):
    # Conteo de bigramas y unigrams
    bigram_counts = {}
    unigram_counts = {}

    # Construir bigramas y contar unigramas
    for sentence in corpus:
        words = sentence.split()
        for i in range(len(words)-1):
            bigram = (words[i], words[i + 1])
            unigram = words[i]

            # Conteo de bigramas
            if bigram in bigram_counts:
                bigram_counts[bigram] += 1
            else:
                bigram_counts[bigram] = 1

            # Conteo de unigrams
            if unigram in unigram_counts:
                unigram_counts[unigram] += 1
            else:
                unigram_counts[unigram] = 1

        # Contar el último unigrama de la oración
        last_word = words[-1]
        if last_word in unigram_counts:
            unigram_counts[last_word] += 1
        else:
            unigram_counts[last_word] = 1

    # Tamaño del vocabulario
    V = len(unigram_counts)

    # Cálculo de las probabilidades suavizadas para bigramas
    add_k_probabilities = {}
    for bigram, bigram_count in bigram_counts.items():
        w_n_1 = bigram[0]  # w_{n-1}
        # Aplicando la ecuación P_Add-k(w_n | w_{n-1}) = (C(w_{n-1}w_n) + k) / (C(w_{n-1}) + kV)
        add_k_probabilities[bigram] = (bigram_count + k) / (unigram_counts[w_n_1] + k * V)

    # Probabilidad para un bigrama no visto
    add_k_probabilities[('<UNK>', '<UNK>')] = k / (V * (V + k))

    return add_k_probabilities


k = 0.05
add_k_prob_bigrams = add_k_smoothing_bigram(corpus, k)
print("\nProbabilidades de bigramas suavizadas con Add-k (k=0.05):")
for bigram, prob in add_k_prob_bigrams.items():
    print(f"P({bigram[1]} | {bigram[0]}) = {prob:.4f}")

k = 0.15
add_k_prob_bigrams = add_k_smoothing_bigram(corpus, k)
print("\nProbabilidades de bigramas suavizadas con Add-k (k=0.15):")
for bigram, prob in add_k_prob_bigrams.items():
    print(f"P({bigram[1]} | {bigram[0]}) = {prob:.4f}")


Probabilidades de bigramas suavizadas con Add-k (k=0.05):
P(models | all) = 0.7241
P(are | models) = 0.8367
P(wrong | are) = 0.4286
P(model | a) = 0.7241
P(is | model) = 0.7241
P(wrong | is) = 0.7241
P(models | some) = 0.7241
P(useful | are) = 0.4286
P(<UNK> | <UNK>) = 0.0006

Probabilidades de bigramas suavizadas con Add-k (k=0.15):
P(models | all) = 0.4894
P(are | models) = 0.6418
P(wrong | are) = 0.3433
P(model | a) = 0.4894
P(is | model) = 0.4894
P(wrong | is) = 0.4894
P(models | some) = 0.4894
P(useful | are) = 0.3433
P(<UNK> | <UNK>) = 0.0018


En la salida anterior se observa que para la probabilidad para k= 0.05, el valor de P( models | a) es 0.0006 y para k=0.15 el valor de
P(models | a) = 0.0018.

### d. Calcule las probabilidades de todos los bigramas y el bigrama no visto a models con back-off y stupid-off:

Referencia: https://github.com/kapumota/Actividades-CC0C2/blob/3d0cde28776954bfc6367f2b94725dffcfaf8f38/Cuadernos-CC0C2/Clase2/Counts-backoff-suavizado.ipynb#L171

In [9]:
from collections import defaultdict
import collections
from typing import List, Tuple

# El vocabulario y el corpus proporcionados
vocabulario = ['<s>', '</s>', 'a', 'all', 'are', 'model', 'models', 'some', 'useful', 'wrong']
corpus = ["all models are wrong", "a model is wrong", "some models are useful"]

# Convertimos el corpus en una lista de listas de tokens
train_corpus = [sentence.split() for sentence in corpus]

# Implementación de modelos N-grama
class NGramModel:
    def __init__(self, n: int):
        self.n = n
        self.ngram_counts = collections.Counter()
        self.context_counts = collections.Counter()
        self.vocab = set()
        self.total_ngrams = 0

    def train(self, corpus: List[List[str]]):
        for document in corpus:
            tokens = ['<s>'] * (self.n - 1) + document + ['</s>']
            self.vocab.update(tokens)
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i + self.n])
                context = tuple(tokens[i:i + self.n - 1])
                self.ngram_counts[ngram] += 1
                self.context_counts[context] += 1
                self.total_ngrams += 1

    def get_ngram_prob(self, ngram: Tuple[str, ...]) -> float:
        count = self.ngram_counts.get(ngram, 0)
        context = ngram[:-1]
        context_count = self.context_counts.get(context, 0)
        if context_count == 0:
            return 0.0
        else:
            return count / context_count

# Implementación de Backoff Estándar
class BackoffNGramModel(NGramModel):
    def __init__(self, n: int, models: List[NGramModel]):
        super().__init__(n)
        self.models = models  # Lista de modelos de diferentes órdenes, ordenados de mayor a menor
        # Actualizamos self.vocab con la unión de los vocabularios de los modelos
        self.vocab = set()
        for model in self.models:
            self.vocab.update(model.vocab)

    def get_ngram_prob(self, ngram: Tuple[str, ...]) -> float:
        for model in self.models:
            ngram_adjusted = ngram[-model.n:]
            prob = model.get_ngram_prob(ngram_adjusted)
            if prob > 0:
                return prob
        # Si ningún modelo tiene el n-grama, retrocedemos al unigrama
        return 0.0

# Implementación del Stupid Backoff
class StupidBackoffNGramModel(NGramModel):
    def __init__(self, n: int, models: List[NGramModel], alpha: float = 0.4):
        super().__init__(n)
        self.models = models  # Lista de modelos de diferentes órdenes, ordenados de mayor a menor
        self.alpha = alpha    # Factor de escala fijo
        # Actualizamos self.vocab con la unión de los vocabularios de los modelos
        self.vocab = set()
        for model in self.models:
            self.vocab.update(model.vocab)

    def get_ngram_prob(self, ngram: Tuple[str, ...]) -> float:
        for i, model in enumerate(self.models):
            ngram_adjusted = ngram[-model.n:]
            prob = model.get_ngram_prob(ngram_adjusted)
            if prob > 0:
                return (self.alpha ** i) * prob
        # Si ningún modelo tiene el n-grama, asignamos probabilidad cero
        return 0.0

# Entrenamiento de los modelos
# Entrenamos modelos base
unigram_model = NGramModel(n=1)
bigram_model = NGramModel(n=2)

unigram_model.train(train_corpus)
bigram_model.train(train_corpus)

# Modelos para backoff estándar
backoff_model = BackoffNGramModel(n=2, models=[bigram_model, unigram_model])

# Modelo Stupid Backoff
stupid_backoff_model = StupidBackoffNGramModel(n=2, models=[bigram_model, unigram_model], alpha=0.4)

# Lista de bigramas del corpus
bigrams_in_corpus = set()
for sentence in train_corpus:
    tokens = ['<s>'] + sentence + ['</s>']
    for i in range(len(tokens) - 1):
        bigr = (tokens[i], tokens[i + 1])
        bigrams_in_corpus.add(bigr)

# Cálculo de probabilidades con back-off para los bigramas del corpus
print("Probabilidades de bigramas con back-off:")
for bigram in bigrams_in_corpus:
    prob = backoff_model.get_ngram_prob(bigram)
    print(f"P('{bigram[1]}' | '{bigram[0]}') = {prob:.4f}")

# Probabilidad del bigrama no visto 'a models' con back-off
prob_a_models_backoff = backoff_model.get_ngram_prob(('a', 'models'))
print(f"\nP('models' | 'a') con back-off = {prob_a_models_backoff:.4f}")

# Cálculo de probabilidades con stupid-backoff para los bigramas del corpus
print("\nProbabilidades de bigramas con stupid-backoff:")
for bigram in bigrams_in_corpus:
    prob = stupid_backoff_model.get_ngram_prob(bigram)
    print(f"P('{bigram[1]}' | '{bigram[0]}') = {prob:.4f}")

# Probabilidad del bigrama no visto 'a models' con stupid-backoff
prob_a_models_stupid = stupid_backoff_model.get_ngram_prob(('a', 'models'))
print(f"\nP('models' | 'a') con stupid-backoff = {prob_a_models_stupid:.4f}")


Probabilidades de bigramas con back-off:
P('wrong' | 'are') = 0.5000
P('model' | 'a') = 1.0000
P('</s>' | 'useful') = 1.0000
P('are' | 'models') = 1.0000
P('wrong' | 'is') = 1.0000
P('some' | '<s>') = 0.3333
P('a' | '<s>') = 0.3333
P('useful' | 'are') = 0.5000
P('is' | 'model') = 1.0000
P('all' | '<s>') = 0.3333
P('models' | 'some') = 1.0000
P('models' | 'all') = 1.0000
P('</s>' | 'wrong') = 1.0000

P('models' | 'a') con back-off = 0.1333

Probabilidades de bigramas con stupid-backoff:
P('wrong' | 'are') = 0.5000
P('model' | 'a') = 1.0000
P('</s>' | 'useful') = 1.0000
P('are' | 'models') = 1.0000
P('wrong' | 'is') = 1.0000
P('some' | '<s>') = 0.3333
P('a' | '<s>') = 0.3333
P('useful' | 'are') = 0.5000
P('is' | 'model') = 1.0000
P('all' | '<s>') = 0.3333
P('models' | 'some') = 1.0000
P('models' | 'all') = 1.0000
P('</s>' | 'wrong') = 1.0000

P('models' | 'a') con stupid-backoff = 0.0533


# Parte 2:

Referencia: https://github.com/kapumota/Actividades-CC0C2/blob/3d0cde28776954bfc6367f2b94725dffcfaf8f38/Cuadernos-CC0C2/Clase2/Modelos-lenguaje2.ipynb#L913

Referencia: https://github.com/kapumota/Actividades-CC0C2/blob/3d0cde28776954bfc6367f2b94725dffcfaf8f38/Cuadernos-CC0C2/Clase2/Topicos-avanzados.ipynb#L501

### e. El suavizado de Good-Turing reasigna la masa de probabilidad de los n-gramas ricos a los n-gramas pobres. Dado un corpus D, supongamos que tratamos todas las unigramas desconocidas como ⟨UNK⟩ por lo tanto, el vocabulario es {w:w∈D}∪{⟨UNK⟩} y N0=1. Calcula r, Nr, para todas las unigramas de la parte 1.

In [2]:
import collections
import math
from typing import List, Tuple, Dict
import numpy as np

corpus = [ "all models are wrong", "a model is wrong",
            "some models are useful"]

In [3]:
def tokenize_corpus(corpus: List[str]) -> List[List[str]]:
    tokenized_corpus = []
    for sentence in corpus:
        tokens = sentence.lower().split()
        tokens = ['<s>'] + tokens + ['</s>']  # Añadimos tokens de inicio y fin de oración
        tokenized_corpus.append(tokens)
    return tokenized_corpus


tokenized_corpus = tokenize_corpus(corpus)

In [4]:
# Conteo de N-gramas
def count_ngrams(tokenized_corpus: List[List[str]], n: int) -> Dict[Tuple[str, ...], int]:
    ngram_counts = collections.Counter()
    for tokens in tokenized_corpus:
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i + n])
            ngram_counts[ngram] += 1
    return ngram_counts

# Contamos unigramas e incluimos <UNK>
unigram_counts = count_ngrams(tokenized_corpus, 1)
unigram_counts[('<UNK>',)] = 0  # Para representar unigramas desconocidos

unigram_counts = count_ngrams(tokenized_corpus, 1)
print("\nConteos de Unigramas:")
for unigram, count in unigram_counts.items():
    print(f"{unigram}: {count}")


Conteos de Unigramas:
('<s>',): 3
('all',): 1
('models',): 2
('are',): 2
('wrong',): 2
('</s>',): 3
('a',): 1
('model',): 1
('is',): 1
('some',): 1
('useful',): 1


In [5]:
bigram_counts = count_ngrams(tokenized_corpus, 2)
trigram_counts = count_ngrams(tokenized_corpus, 3)

#  Cálculo de N(C)
def calculate_NC(ngram_counts: Dict[Tuple[str, ...], int]) -> Dict[int, int]:
    count_of_counts = collections.Counter()
    for count in ngram_counts.values():
        count_of_counts[count] += 1
    return count_of_counts

def sort_NC(NC: Dict[int, int]) -> Tuple[np.ndarray, np.ndarray]:
    counts = np.array(list(NC.keys()))
    frequencies = np.array([NC[count] for count in counts])
    sorted_indices = np.argsort(counts)
    return counts[sorted_indices], frequencies[sorted_indices]

In [6]:
# Calculamos N(C) para bigramas (puede aplicarse a unigramas y trigramas de manera similar)
NC_bigram = calculate_NC(bigram_counts)
counts_bigram, frequencies_bigram = sort_NC(NC_bigram)

In [7]:
# Suavizado Good-Turing
def good_turing_discounting(ngram_counts: Dict[Tuple[str, ...], int]) -> Dict[Tuple[str, ...], float]:
    # Calculamos N(C)
    NC = calculate_NC(ngram_counts)
    counts, frequencies = sort_NC(NC)

    # Ajuste de conteos
    total_ngrams = sum(ngram_counts.values())
    max_count = max(counts)
    adjusted_counts = {}

    for ngram, count in ngram_counts.items():
        if count < max_count:
            Nc = NC[count]
            Nc1 = NC.get(count + 1, 0)
            if Nc > 0:
                C_star = (count + 1) * (Nc1 / Nc)
                adjusted_counts[ngram] = C_star
            else:
                adjusted_counts[ngram] = count
        else:
            adjusted_counts[ngram] = count  # Para conteos máximos, no ajustamos
    return adjusted_counts

In [8]:
#Cálculo de probabilidades ajustadas
def calculate_probabilities(adjusted_counts: Dict[Tuple[str, ...], float],
                            n_minus1_counts: Dict[Tuple[str, ...], int]) -> Dict[Tuple[str, ...], float]:
    probabilities = {}
    for ngram, adjusted_count in adjusted_counts.items():
        context = ngram[:-1]
        context_count = n_minus1_counts.get(context, sum(n_minus1_counts.values()))
        probability = adjusted_count / context_count if context_count > 0 else 0.0
        probabilities[ngram] = probability
    return probabilities

In [9]:
adjusted_bigram_counts = good_turing_discounting(bigram_counts)

adjusted_unigram_counts = good_turing_discounting(unigram_counts)

# Calculamos las probabilidades ajustadas para bigramas
unigram_total_count = sum(unigram_counts.values())
bigram_probabilities = calculate_probabilities(adjusted_bigram_counts, unigram_counts)

In [10]:
# Asignación de probabilidad a N-gramas no observados
def probability_of_unseen(NC: Dict[int, int], total_ngrams: int) -> float:
    N1 = NC.get(1, 0)
    return N1 / total_ngrams if total_ngrams > 0 else 0.0

# Probabilidad de n-gramas no observados para bigramas
total_bigrams = sum(bigram_counts.values())
P_unseen_bigram = probability_of_unseen(NC_bigram, total_bigrams)


In [11]:
#Uso del modelo para calcular la probabilidad de una oración
def sentence_probability(sentence: str, bigram_probabilities: Dict[Tuple[str, str], float], P_unseen: float) -> float:
    tokens = ['<s>'] + sentence.lower().split() + ['</s>']
    probability_log_sum = 0.0
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i + 1])
        prob = bigram_probabilities.get(bigram, P_unseen)
        probability_log_sum += math.log(prob) if prob > 0 else float('-inf')
    return math.exp(probability_log_sum)

In [12]:
NC = calculate_NC(unigram_counts)
NC[0] = 1  # Agregamos N0 = 1 para el caso <UNK>
counts_unigram, frequencies_unigram  = sort_NC(NC)

print("\n r (veces que ocurre un unigrama):\n")
for unigram, count in unigram_counts.items():
    print(f"{unigram}: {count}")

print("\n Nr (número de unigramas que ocurren exactamente r veces):\n")
print(NC)


 r (veces que ocurre un unigrama):

('<s>',): 3
('all',): 1
('models',): 2
('are',): 2
('wrong',): 2
('</s>',): 3
('a',): 1
('model',): 1
('is',): 1
('some',): 1
('useful',): 1

 Nr (número de unigramas que ocurren exactamente r veces):

Counter({1: 6, 2: 3, 3: 2, 0: 1})


### f. Para r<3, calcula cr y las probabilidades de todas las unigramas.

In [13]:
# Mostramos las probabilidades para r < 3
print("\nProbabilidades ajustadas para r < 3:")
for unigram, count in adjusted_unigram_counts.items():
    if unigram_counts[unigram] < 3:
        print(f"C{unigram}: {count}")


Probabilidades ajustadas para r < 3:
C('all',): 1.0
C('models',): 2.0
C('are',): 2.0
C('wrong',): 2.0
C('a',): 1.0
C('model',): 1.0
C('is',): 1.0
C('some',): 1.0
C('useful',): 1.0


In [14]:
unigram_probabilities = calculate_probabilities(adjusted_unigram_counts, {'<s>': unigram_total_count})

# Mostramos las probabilidades para r < 3
print("\nProbabilidades ajustadas para r < 3:")
for unigram, prob in unigram_probabilities.items():
    if unigram_counts[unigram] < 3:
        print(f"P{unigram}: {prob}")


Probabilidades ajustadas para r < 3:
P('all',): 0.05555555555555555
P('models',): 0.1111111111111111
P('are',): 0.1111111111111111
P('wrong',): 0.1111111111111111
P('a',): 0.05555555555555555
P('model',): 0.05555555555555555
P('is',): 0.05555555555555555
P('some',): 0.05555555555555555
P('useful',): 0.05555555555555555


### g. Para el valor máximo de r, Nr+1=0. En este caso, la probabilidad P(w: #w=r) aún puede estimarse mediante MLE (Máxima Verosimilitud). Calcula la   probabilidad de las unigramas de la parte 1 que aparecen con mayor frecuencia, es decir, r=3.

In [15]:
total_tokens = sum(unigram_counts.values())
probabilities_g = {}
print("\nProbabilidades para unigramas con r = 3 usando MLE:")
for unigram, count in unigram_counts.items():
    if count == 3:
        probability = count / total_tokens
        unigram_probabilities[unigram] = probability
        print(f"C{unigram}: {count}, P(w) = {probability:.4f}")


Probabilidades para unigramas con r = 3 usando MLE:
C('<s>',): 3, P(w) = 0.1667
C('</s>',): 3, P(w) = 0.1667


### h. Muestra que la suma de las probabilidades de todas las unigramas dadas en (f) y (g) no es 1. Intenta normalizar las probabilidades.

In [16]:
NC_unigram = calculate_NC(unigram_counts)
total_unigrams = sum(unigram_counts.values())
P_unseen_unigram = probability_of_unseen(NC_unigram, total_unigrams)

In [17]:
probabilities_normalized = {w: p / (sum(unigram_probabilities.values())+P_unseen_unigram) for w, p in unigram_probabilities.items()}

suma_prob = 0
print("\nProbabilidades normalizadas:")
for unigram, prob in probabilities_normalized.items():
    print(f"{unigram}: P(w) = {prob:.4f}")
    suma_prob += prob


Probabilidades normalizadas:
('<s>',): P(w) = 0.1250
('all',): P(w) = 0.0417
('models',): P(w) = 0.0833
('are',): P(w) = 0.0833
('wrong',): P(w) = 0.0833
('</s>',): P(w) = 0.1250
('a',): P(w) = 0.0417
('model',): P(w) = 0.0417
('is',): P(w) = 0.0417
('some',): P(w) = 0.0417
('useful',): P(w) = 0.0417


A continuación se observa que la suma de las probabilidades vistas en f y g no suman 1.

In [29]:
print(suma_prob)

0.7499999999999998


### i. En un corpus grande, Nr puede ser cero para valores grandes de r. Esto puede ser problemático, ya que conduce a que los valores estimados de P(w:#w=r) sean cero. Una forma de resolver este problema es usar una línea suavizada para ajustar aproximadamente la distribución de los valores conocidos. Supongamos que cambiamos la segunda oración de ejemplo de la parte 1 a ‘a model is wrong wrong wrong wrong’ de modo que N4=1, pero S_5=0. Adivina un buen valor suavizado de N_5. Usa el valor aproximado de N_5 y el valor original de los otros recuentos de frecuencia para calcular las probabilidades de todas las unigramas.

In [18]:
# Actualizamos el corpus
corpus_i = [
    "all models are wrong",
    "a model is wrong wrong wrong wrong",
    "some models are useful"
]

# Tokenizamos y contamos nuevamente
tokenized_corpus_i = tokenize_corpus(corpus_i)
unigram_counts_i = count_ngrams(tokenized_corpus_i, 1)
NC_unigram_i = calculate_NC(unigram_counts_i)

# Mostramos los nuevos valores de N_r
print("\nr y N_r:")
for r, Nr in sorted(NC_unigram_i.items()):
    print(f"r = {r}, N_r = {Nr}")



r y N_r:
r = 1, N_r = 6
r = 2, N_r = 2
r = 3, N_r = 2
r = 5, N_r = 1


In [19]:
# Función de suavizado Good-Turing con valor estimado de N_r
def good_turing_discounting_unigrams_smoothed(unigram_counts: Dict[Tuple[str, ...], int], N4_estimate: float) -> Dict[Tuple[str, ...], float]:
    NC = calculate_NC(unigram_counts)
    # Estimamos N_4
    NC[4] = N4_estimate
    adjusted_counts = {}
    counts = sorted(NC.keys())
    max_r = max(counts)

    for unigram, count in unigram_counts.items():
        r = count
        Nr = NC.get(r, 0)
        Nr1 = NC.get(r + 1, 0)
        if Nr > 0 and Nr1 > 0:
            c_star = (r + 1) * (Nr1 / Nr)
        else:
            c_star = r  # Si Nr o Nr1 es 0, no ajustamos
        adjusted_counts[unigram] = c_star
    return adjusted_counts

# Aplicamos el suavizado con N4_estimate = 0.5
adjusted_unigram_counts_i = good_turing_discounting_unigrams_smoothed(unigram_counts_i, N4_estimate=0.5)

# Calculamos las probabilidades ajustadas
unigram_probabilities_i = calculate_probabilities(adjusted_unigram_counts_i, {'<s>': unigram_total_count})

# Normalizamos las probabilidades
total_probability_i = sum(unigram_probabilities_i.values())
if total_probability_i != 1.0:
    for unigram in unigram_probabilities_i:
        unigram_probabilities_i[unigram] /= total_probability_i

# Mostramos las probabilidades finales
print("\nProbabilidades ajustadas tras el suavizado:")
for unigram, prob in unigram_probabilities_i.items():
    print(f"{unigram}: {prob}")



Probabilidades ajustadas tras el suavizado:
('<s>',): 0.05882352941176472
('all',): 0.03921568627450981
('models',): 0.17647058823529416
('are',): 0.17647058823529416
('wrong',): 0.2941176470588236
('</s>',): 0.05882352941176472
('a',): 0.03921568627450981
('model',): 0.03921568627450981
('is',): 0.03921568627450981
('some',): 0.03921568627450981
('useful',): 0.03921568627450981


## Parte 3

## Implementación inicial del Brown Clustering

In [ ]:
import urllib.request
import json

articulos = [
    "Inteligencia artificial", "Aprendizaje automático", "Biotecnología"
]

# URL de la API de Wikipedia para obtener el extracto en formato JSON
base_url = 'https://es.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&explaintext&titles={}'

# Archivo para guardar el corpus
filename = 'corpus.txt'

with open(filename, 'w', encoding='utf-8') as file:
    for title in articulos:
        """
        Descarga el contenido de artículos de Wikipedia y los guarda en un archivo de texto.

        Args:
            title (str): El título del artículo de Wikipedia a descargar.

        Funcionalidad:
            - Codifica el título del artículo para incluirlo en una URL de solicitud a la API de Wikipedia.
            - Descarga el contenido del artículo en formato JSON y extrae el campo 'extract', que contiene el texto del artículo.
            - Si el extracto no está vacío, lo escribe en el archivo especificado y confirma la descarga en consola.
            - Si el extracto está vacío, indica que el artículo no se pudo descargar.

        Output:
            Archivo 'corpus.txt' con el contenido de todos los artículos descargados.
        """

        # Codificar el título para URL
        encoded_title = urllib.request.quote(title)
        url = base_url.format(encoded_title)
        print(f"Descargando el artículo: {title}")
        response = urllib.request.urlopen(url)
        data = response.read().decode('utf-8')

        # Extraer el texto del JSON
        json_data = json.loads(data)
        pages = json_data['query']['pages']
        page = next(iter(pages.values()))
        extract = page.get('extract', '')

        # Escribir el texto en el archivo si el extracto no está vacío
        if extract:
            file.write(extract + '\n')
            print(f"Artículo '{title}' descargado.")
        else:
            print(f"No se pudo descargar el artículo '{title}'.")

print(f"Todos los artículos han sido descargados y guardados en '{filename}'.")

Descargando el artículo: Inteligencia artificial


In [35]:
import string
from collections import Counter

def tokenize(text):
    """
    Convierte el texto en minúsculas, remueve la puntuación y lo tokeniza en palabras.

    Args:
        text (str): Texto a procesar.

    Returns:
        list: Lista de palabras (tokens) extraídas del texto.
    """
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    return tokens

def stem(word):
    """
    Aplica una reducción simple de sufijos comunes en español para simular el stemming.

    Args:
        word (str): Palabra a la que se le aplicará el stemming.

    Returns:
        str: La palabra sin sufijos comunes.
    """
    suffixes = ['aciones', 'imientos', 'amiento', 'ición', 'adora', 'ación', 'adoras', 'adores', 'ante',
                'ancia', 'mente', 'idad', 'ivas', 'ivos', 'anza', 'icos', 'icas', 'ico', 'ica',
                'oso', 'osa', 'osos', 'osas', 'ismo', 'ismos', 'able', 'ables', 'ible', 'ibles',
                'ente', 'entes', 'mente']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

stopwords = set([
     'vuestro', 'vuestros', 'y', 'ya', 'yo', 'él', 'éramos','a', 'al', 'algo', 'algunas', 'algunos',
     'ante', 'antes', 'como', 'con', 'contra', 'de', 'del', 'desde', 'donde', 'durante', 'e', 'el',
     'ella', 'ellas', 'ellos', 'en', 'entre', 'era', 'erais', 'eran', 'eras', 'eres', 'es',
     'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas',
    'vosotras', 'vosotros', 'vuestra', 'vuestras'
])


def preprocess_corpus(filename):
    """
    Lee un archivo de texto, realiza tokenización, stemming, remueve stopwords y filtra palabras raras.

    Args:
        filename (str): Ruta del archivo de texto a procesar.

    Returns:
        list: Lista de tokens procesados que cumplen con un umbral de frecuencia.
    """
    print("Iniciando el preprocesamiento del corpus")
    with open(filename, 'r', encoding='utf-8') as file:
        word_counter = Counter()
        final_tokens = []
        for line in file:
            tokens = tokenize(line)
            stems = [stem(token) for token in tokens]
            tokens_filtered = [word for word in stems if word not in stopwords]
            word_counter.update(tokens_filtered)
            final_tokens.extend(tokens_filtered)
            
    threshold = 5
    frequent_words = {word for word, count in word_counter.items() if count >= threshold}
    final_tokens = [word for word in final_tokens if word in frequent_words]

    print(f"Tamaño del vocabulario final: {len(set(final_tokens))} palabras")
    return final_tokens

final_tokens = preprocess_corpus('corpus.txt')

Iniciando el preprocesamiento del corpus
Tamaño del vocabulario final: 477 palabras


In [36]:
def brown_clustering_initial(final_tokens):
    """
    Brown Clustering.

    Pasos:
    1. Inicialización: asigne cada palabra a su propio grupo.

    2. Cálculos de probabilidad: estime P(c) y P(c_i, c_j).

    3. Búsqueda de combinación óptima: evalúe ΔI(c_i, c_j) para cada par.

    4. Combinación de grupos: combine los grupos que minimicen ΔI.

    5. Repita hasta alcanzar la cantidad deseada de grupos.

    """
    # Inicialización
    vocabulary = set(final_tokens)
    clusters = {}  # cluster_id: conjunto de palabras
    word_to_cluster = {}  # word: cluster_id

    for i, word in enumerate(vocabulary):
        clusters[i] = set([word])
        word_to_cluster[word] = i

    # Cáculo de probabilidades
    from collections import defaultdict
    import math

    # Cuenta los unigramas y bigramas
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)

    for i in range(len(final_tokens) - 1):
        w1 = final_tokens[i]
        w2 = final_tokens[i + 1]
        bigram_counts[(w1, w2)] += 1
        unigram_counts[w1] += 1
    unigram_counts[final_tokens[-1]] += 1  # última palabra

    total_bigrams = sum(bigram_counts.values())
    total_words = sum(unigram_counts.values())

    cluster_counts = defaultdict(int) 
    cluster_bigram_counts = defaultdict(int) 

    for word, count in unigram_counts.items():
        c = word_to_cluster[word]
        cluster_counts[c] += count

    for (w1, w2), count in bigram_counts.items():
        c1 = word_to_cluster[w1]
        c2 = word_to_cluster[w2]
        cluster_bigram_counts[(c1, c2)] += count

    # Caclula la información mutua
    def compute_mutual_information(cluster_counts, cluster_bigram_counts, total_bigrams, total_words):
        I = 0.0
        for (c_i, c_j), count in cluster_bigram_counts.items():
            p_ci_cj = count / total_bigrams
            p_ci = cluster_counts[c_i] / total_words
            p_cj = cluster_counts[c_j] / total_words
            if p_ci_cj > 0 and p_ci > 0 and p_cj > 0:
                I += p_ci_cj * math.log(p_ci_cj / (p_ci * p_cj))
        return I

    I_current = compute_mutual_information(cluster_counts, cluster_bigram_counts, total_bigrams, total_words)

    # Loop 
    desired_num_clusters = 100 
    num_clusters = len(clusters)

    while num_clusters > desired_num_clusters:
        min_delta_I = None
        best_pair = None
        best_cluster_counts = None
        best_cluster_bigram_counts = None
        best_clusters = None
        best_word_to_cluster = None

        cluster_ids = list(clusters.keys())

        for idx_i in range(len(cluster_ids)):
            c_i = cluster_ids[idx_i]
            for idx_j in range(idx_i + 1, len(cluster_ids)):
                c_j = cluster_ids[idx_j]

                # Crea clusters temporales
                clusters_temp = clusters.copy()
                clusters_temp[c_i] = clusters[c_i].union(clusters[c_j])
                del clusters_temp[c_j]

                # Actualiza word_to_cluster
                word_to_cluster_temp = word_to_cluster.copy()
                for word in clusters[c_j]:
                    word_to_cluster_temp[word] = c_i

                # Actualiza cluster_counts_temp
                cluster_counts_temp = cluster_counts.copy()
                cluster_counts_temp[c_i] += cluster_counts_temp[c_j]
                del cluster_counts_temp[c_j]

                # Actualiza cluster_bigram_counts_temp
                cluster_bigram_counts_temp = defaultdict(int)
                for (ci, cj), count in cluster_bigram_counts.items():
                    ci_new = ci
                    cj_new = cj
                    if ci == c_j:
                        ci_new = c_i
                    if cj == c_j:
                        cj_new = c_i
                    cluster_bigram_counts_temp[(ci_new, cj_new)] += count

                # Calula la información mutua
                I_merge = compute_mutual_information(cluster_counts_temp, cluster_bigram_counts_temp, total_bigrams, total_words)

                delta_I = I_merge - I_current

                if min_delta_I is None or delta_I < min_delta_I:
                    min_delta_I = delta_I
                    best_pair = (c_i, c_j)
                    best_cluster_counts = cluster_counts_temp
                    best_cluster_bigram_counts = cluster_bigram_counts_temp
                    best_clusters = clusters_temp
                    best_word_to_cluster = word_to_cluster_temp

        # Une los mejores pares
        c_i, c_j = best_pair
        clusters = best_clusters
        cluster_counts = best_cluster_counts
        cluster_bigram_counts = best_cluster_bigram_counts
        word_to_cluster = best_word_to_cluster
        I_current += min_delta_I
        num_clusters -= 1
        print(f"Se unieron los clústeres {c_i} y {c_j}; número de clústeres: {num_clusters}")

    return clusters

In [37]:
import time
import tracemalloc

start_time = time.time()
tracemalloc.start()

clusters_initial = brown_clustering_initial(final_tokens)

current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

tracemalloc.stop()

print(f"Tiempo de ejecución de la implementación inicial: {end_time - start_time} segundos")
print(f"Uso de memoria de la implementación inicial: Actual={current / 10**6} MB; Pico={peak / 10**6} MB")

KeyboardInterrupt: 

In [ ]:
def calculate_silhouette_coefficient(clusters, word_to_cluster, bigram_counts):
    """
    Calcula el coeficiente de silhouette para evaluar la calidad del clustering
    """
    from collections import defaultdict

    # Calcular la distancia media entre grupos a(i)
    a_values = {}
    for cluster_id, words in clusters.items():
        for word in words:
            # Calcular la distancia promedio a otras palabras en el mismo grupo
            intra_distances = []
            for other_word in words:
                if word != other_word:
                    distance = 1 - (bigram_counts.get((word, other_word), 0) + bigram_counts.get((other_word, word), 0))
                    intra_distances.append(distance)
            a_values[word] = sum(intra_distances) / (len(words) - 1) if len(words) > 1 else 0

    # Calcular la distancia promedio del grupo más cercano b(i)
    b_values = {}
    for word in word_to_cluster:
        cluster_id = word_to_cluster[word]
        min_distance = None
        for other_cluster_id, words in clusters.items():
            if other_cluster_id != cluster_id:
                inter_distances = []
                for other_word in words:
                    distance = 1 - (bigram_counts.get((word, other_word), 0) + bigram_counts.get((other_word, word), 0))
                    inter_distances.append(distance)
                average_distance = sum(inter_distances) / len(inter_distances)
                if min_distance is None or average_distance < min_distance:
                    min_distance = average_distance
        b_values[word] = min_distance

    # Calcular puntuaciones de silueta
    s_values = {}
    for word in word_to_cluster:
        a = a_values[word]
        b = b_values[word]
        s = (b - a) / max(a, b) if max(a, b) > 0 else 0
        s_values[word] = s

    # Devuelve la puntuación media de silueta
    overall_silhouette = sum(s_values.values()) / len(s_values)
    return overall_silhouette

In [ ]:
def brown_clustering_improved(final_tokens):
    """
    Implementación mejorada de Brown Clustering.

    Mejoras:
    - Utiliza estructuras de datos eficientes: utiliza colas de prioridad para gestionar pares de clústeres y su ΔI.
    - Implementa técnicas de poda: limita las fusiones de candidatos mediante clústeres vecinos.

    """
    # Inicialización
    vocabulary = set(final_tokens)
    clusters = {}  
    word_to_cluster = {}  

    for i, word in enumerate(vocabulary):
        clusters[i] = set([word])
        word_to_cluster[word] = i

    # Cálculo de probabilidades
    from collections import defaultdict
    import math
    import heapq

    # Contar bigramas y unigramas
    bigram_counts = defaultdict(int)
    unigram_counts = defaultdict(int)

    for i in range(len(final_tokens) - 1):
        w1 = final_tokens[i]
        w2 = final_tokens[i + 1]
        bigram_counts[(w1, w2)] += 1
        unigram_counts[w1] += 1
    unigram_counts[final_tokens[-1]] += 1  # última palabra

    total_bigrams = sum(bigram_counts.values())
    total_words = sum(unigram_counts.values())

    # Inicializar cluster_counts y cluster_bigram_counts
    cluster_counts = defaultdict(int)  
    cluster_bigram_counts = defaultdict(int) 

    for word, count in unigram_counts.items():
        c = word_to_cluster[word]
        cluster_counts[c] += count

    for (w1, w2), count in bigram_counts.items():
        c1 = word_to_cluster[w1]
        c2 = word_to_cluster[w2]
        cluster_bigram_counts[(c1, c2)] += count

    # Inicializar clústeres vecinos
    neighbors = defaultdict(set)  
    for (c1, c2) in cluster_bigram_counts.keys():
        if c1 != c2:
            neighbors[c1].add(c2)
            neighbors[c2].add(c1)

    # Inicializar el montón con ΔI para cada par de vecinos
    heap = []

    def compute_delta_I(c_i, c_j):
        # Cálculo eficiente de ΔI basado en recuentos de clústeres y recuentos de bigramas
        delta_I = 0.0
        # Calcular P(c_i), P(c_j), P(c_i, c_j)
        p_ci = cluster_counts[c_i] / total_words
        p_cj = cluster_counts[c_j] / total_words
        p_ci_cj = cluster_bigram_counts.get((c_i, c_j), 0) / total_bigrams

        if p_ci_cj > 0 and p_ci > 0 and p_cj > 0:
            delta_I = p_ci_cj * math.log(p_ci_cj / (p_ci * p_cj))
        else:
            delta_I = 0.0
        return delta_I

    for c_i in clusters.keys():
        for c_j in neighbors[c_i]:
            if c_i < c_j:  # Evitar duplicados
                delta_I = compute_delta_I(c_i, c_j)
                heapq.heappush(heap, (delta_I, c_i, c_j))

    desired_num_clusters = 100  
    num_clusters = len(clusters)

    while num_clusters > desired_num_clusters and heap:
        delta_I, c_i, c_j = heapq.heappop(heap)

        # Verificar si los clústeres se han fusionado
        if c_i not in clusters or c_j not in clusters:
            continue  # Omitir si los clústeres ya no existen

        # Fusionar los clústeres c_i y c_j en c_new
        c_new = c_i  # Utilice c_i como el nuevo ID del clúster

        clusters[c_new].update(clusters[c_j])
        del clusters[c_j]

        # Actualiza word_to_cluster
        for word in clusters[c_new]:
            word_to_cluster[word] = c_new

        # Actualiza cluster_counts
        cluster_counts[c_new] += cluster_counts[c_j]
        del cluster_counts[c_j]

        # Actualiza cluster_bigram_counts
        cluster_bigram_counts_updated = defaultdict(int)
        for (ci, cj), count in cluster_bigram_counts.items():
            ci_new = ci
            cj_new = cj
            if ci == c_j:
                ci_new = c_new
            if cj == c_j:
                cj_new = c_new
            cluster_bigram_counts_updated[(ci_new, cj_new)] += count
        cluster_bigram_counts = cluster_bigram_counts_updated

        # Actualiza neighbors
        neighbors[c_new].update(neighbors[c_j])
        neighbors[c_new].discard(c_new)
        del neighbors[c_j]
        for neighbor in neighbors:
            if c_j in neighbors[neighbor]:
                neighbors[neighbor].discard(c_j)
                neighbors[neighbor].add(c_new)

        # Actualiza heap
        for c_k in neighbors[c_new]:
            if c_k != c_new:
                delta_I = compute_delta_I(c_new, c_k)
                heapq.heappush(heap, (delta_I, c_new, c_k))

        num_clusters -= 1
        print(f"Se unieron los clústeres {c_i} y {c_j}; número de clústeres: {num_clusters}")

    # Llamar a la función calculate_silhouette_coefficient
    silhouette_score = calculate_silhouette_coefficient(clusters, word_to_cluster, bigram_counts)
    print(f"Coeficiente de Silhouette para la implementación con mejoras: {silhouette_score}")

    return clusters


In [ ]:
inicio = time.time()
tracemalloc.start()

clusters_improved = brown_clustering_improved(final_tokens)

current, peak = tracemalloc.get_traced_memory()
fin = time.time()

tracemalloc.stop()

print(f"Tiempo de implementación con mejoras: {fin - inicio} segundos")
print(f"Uso mejorado de la memoria de implementación: Actual={current / 10**6} MB; Pico={peak / 10**6} MB")

# LSA

## Pre procesamiento del corpus

In [103]:
import requests

def get_random_wikipedia_articles(num_articles):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"
    articles = []

    while len(articles) < num_articles:
        PARAMS = {
            "format": "json",
            "action": "query",
            "list": "random",
            "rnnamespace": 0,  
            "rnlimit": min(500, num_articles - len(articles)),  
        }

        response = S.get(url=URL, params=PARAMS)
        data = response.json()
        random_articles = data['query']['random']
        articles.extend([item['title'] for item in random_articles])

    return articles

# Obtener 1000 artículos aleatorios
articulos_lsa= get_random_wikipedia_articles(100000)

In [108]:
import string
from collections import Counter

def tokenize(text):
    """
    Convierte el texto en minúsculas, remueve la puntuación y lo tokeniza en palabras.

    Args:
        text (str): Texto a procesar.

    Returns:
        list: Lista de palabras (tokens) extraídas del texto.
    """
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    return tokens

def stem(word):
    """
    Aplica una reducción simple de sufijos comunes en español para simular el stemming.

    Args:
        word (str): Palabra a la que se le aplicará el stemming.

    Returns:
        str: La palabra sin sufijos comunes.
    """
    suffixes = ['aciones', 'imientos', 'amiento', 'ición', 'adora', 'ación', 'adoras', 'adores', 'ante',
                'ancia', 'mente', 'idad', 'ivas', 'ivos', 'anza', 'icos', 'icas', 'ico', 'ica',
                'oso', 'osa', 'osos', 'osas', 'ismo', 'ismos', 'able', 'ables', 'ible', 'ibles',
                'ente', 'entes', 'mente']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

stopwords = set([
    'vuestro', 'vuestros', 'y', 'ya', 'yo', 'él', 'éramos', 'a', 'al', 'algo', 'algunas', 'algunos',
    'ante', 'antes', 'como', 'con', 'contra', 'de', 'del', 'desde', 'donde', 'durante', 'e', 'el',
    'ella', 'ellas', 'ellos', 'en', 'entre', 'era', 'erais', 'eran', 'eras', 'eres', 'es',
    'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas',
    'vosotras', 'vosotros', 'vuestra', 'vuestras'
])

def preprocess_corpus(filename, articulos):
    """
    Lee un archivo de texto, realiza tokenización, stemming, remueve stopwords y filtra palabras raras.
    También separa el corpus en documentos individuales basados en los títulos de los artículos.

    Args:
        filename (str): Ruta del archivo de texto a procesar.
        articulos (list): Lista de títulos de los artículos.

    Returns:
        tuple: (documents, labels)
            - documents: Lista de documentos, cada uno es una lista de tokens procesados.
            - labels: Lista de etiquetas correspondientes a cada documento.
    """
    print("Iniciando el preprocesamiento del corpus")
    documents = []
    labels = []
    word_counter = Counter()

    # Leer todo el contenido del archivo
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # Dividir el contenido en documentos utilizando doble salto de línea como separador
    raw_documents = content.strip().split('\n\n')  # Ajustar según el formato real del archivo

    # Asegurar que el número de documentos coincide con el número de títulos
    min_length = min(len(raw_documents), len(articulos))
    raw_documents = raw_documents[:min_length]
    
    articulos = articulos[:min_length]

    for idx, text in enumerate(raw_documents):
        # Tokenización y preprocesamiento
        tokens = tokenize(text)
        stems = [stem(token) for token in tokens]
        tokens_filtered = [word for word in stems if word not in stopwords]
        word_counter.update(tokens_filtered)
        documents.append(tokens_filtered)
        labels.append(articulos[idx])

    # Filtrado de palabras raras
    threshold = 3
    frequent_words = {word for word, count in word_counter.items() if count >= threshold}

    # Actualizar los documentos con las palabras frecuentes
    final_documents = []
    for doc in documents:
        doc_filtered = [word for word in doc if word in frequent_words]
        final_documents.append(doc_filtered)

    print(f"Tamaño del vocabulario final: {len(frequent_words)} palabras")
    return final_documents, labels

In [109]:
# Preprocesar el corpus y obtener documentos y etiquetas
documents, labels = preprocess_corpus(filename, articulos_lsa)

Iniciando el preprocesamiento del corpus
Tamaño del vocabulario final: 16821 palabras


## Implementación inicial

In [110]:
def lsa_initial(documents, k):
    """
    Análisis Semántico Latente (LSA) implementación inicial.

    Pasos:
    1. Construir la matriz término-documento X con pesos TF-IDF.
    2. Implementar SVD para descomponer X en U, Σ, V^T.
    3. Reducción de Dimensionalidad: Seleccionar los k valores singulares más grandes y sus vectores correspondientes.
    4. Proyección: Representar términos y documentos en el espacio reducido.

    Parámetros:
    - documents: Lista de documentos, cada documento es una lista de tokens.
    - k: Número de dimensiones para reducir.

    Retorna:
    - term_vectors: Vectores de términos en dimensionalidad reducida.
    - document_vectors: Vectores de documentos en dimensionalidad reducida.
    - term_index: Diccionario que mapea términos a índices.
    """
    import numpy as np
    import math
    from collections import defaultdict

    # Construir la matriz término-documento X con pesos TF-IDF.
    # Construir vocabulario
    vocabulary = set()
    for doc in documents:
        vocabulary.update(doc)
    vocabulary = list(vocabulary)
    term_index = {term: idx for idx, term in enumerate(vocabulary)}
    num_terms = len(vocabulary)
    num_docs = len(documents)

    # Inicializar matriz término-documento
    X = np.zeros((num_terms, num_docs))

    # Calcular frecuencias de términos y frecuencias de documentos
    df = defaultdict(int)  # Frecuencia de documentos
    for idx, doc in enumerate(documents):
        term_counts = defaultdict(int)
        for term in doc:
            term_counts[term] += 1
        for term, count in term_counts.items():
            term_idx = term_index[term]
            X[term_idx, idx] = count  # Frecuencia de término
        for term in set(doc):
            df[term] += 1

    # Calcular IDF y aplicar ponderación TF-IDF
    idf = {}
    for term in vocabulary:
        idf[term] = math.log(num_docs / (1 + df[term]))
    for term, idx in term_index.items():
        X[idx, :] = X[idx, :] * idf[term]

    # Implementar SVD
    U, Sigma, VT = np.linalg.svd(X, full_matrices=False)

    # Reducción de Dimensionalidad
    U_k = U[:, :k]
    Sigma_k = np.diag(Sigma[:k])
    VT_k = VT[:k, :]

    # Proyección
    term_vectors = np.dot(U_k, Sigma_k)
    document_vectors = np.dot(Sigma_k, VT_k)

    return term_vectors, document_vectors.T, term_index


In [111]:
def lsa_improved(documents, k, iterations=5):
    """
    Análisis Semántico Latente (LSA) implementación mejorada usando método iterativo.

    Mejoras:
    - Implementar SVD truncado usando el método de potencia para calcular los k valores y vectores singulares más significativos.
    - Optimizar operaciones usando matrices dispersas.

    Parámetros:
    - documents: Lista de documentos, cada documento es una lista de tokens.
    - k: Número de dimensiones para reducir.
    - iterations: Número de iteraciones para el método de potencia.

    Retorna:
    - term_vectors: Vectores de términos en dimensionalidad reducida.
    - document_vectors: Vectores de documentos en dimensionalidad reducida.
    - term_index: Diccionario que mapea términos a índices.
    """
    import numpy as np
    import math
    from collections import defaultdict
    from scipy.sparse import csc_matrix

    # Construir la matriz término-documento X con pesos TF-IDF.
    # Construir vocabulario
    vocabulary = set()
    for doc in documents:
        vocabulary.update(doc)
    vocabulary = list(vocabulary)
    term_index = {term: idx for idx, term in enumerate(vocabulary)}
    num_terms = len(vocabulary)
    num_docs = len(documents)

    # Inicializar datos para matriz dispersa
    data = []
    rows = []
    cols = []

    # Calcular frecuencias de términos y frecuencias de documentos
    df = defaultdict(int)  # Frecuencia de documentos
    for idx, doc in enumerate(documents):
        term_counts = defaultdict(int)
        for term in doc:
            term_counts[term] += 1
        for term, count in term_counts.items():
            term_idx = term_index[term]
            data.append(count)
            rows.append(term_idx)
            cols.append(idx)
        for term in set(doc):
            df[term] += 1

    # Crear matriz término-documento dispersa
    X = csc_matrix((data, (rows, cols)), shape=(num_terms, num_docs))

    # Calcular IDF y aplicar ponderación TF-IDF
    idf = {}
    for term in vocabulary:
        idf[term] = math.log(num_docs / (1 + df[term]))
    for term, idx in term_index.items():
        X[idx, :] = X[idx, :].multiply(idf[term])

    # Implementar SVD truncado usando el método de potencia
    def truncated_svd(X, k, iterations):
        n, m = X.shape
        V = np.random.rand(m, k)
        for _ in range(iterations):
            U = X.dot(V)
            U, _ = np.linalg.qr(U)
            V = X.T.dot(U)
            V, _ = np.linalg.qr(V)
        Sigma = np.diag(np.linalg.norm(X.dot(V), axis=0))
        return U, Sigma, V.T

    U_k, Sigma_k, VT_k = truncated_svd(X, k, iterations)

    # Proyección
    term_vectors = U_k.dot(Sigma_k)
    document_vectors = VT_k.T.dot(Sigma_k)

    return term_vectors, document_vectors, term_index


In [112]:
def classify_documents(document_vectors_train, labels_train, document_vectors_test, k=3):
    """
    Clasificar documentos usando el algoritmo k-NN.

    Parámetros:
    - document_vectors_train: Array de vectores de documentos de entrenamiento.
    - labels_train: Lista de etiquetas para los documentos de entrenamiento.
    - document_vectors_test: Array de vectores de documentos a clasificar.
    - k: Número de vecinos a considerar.

    Retorna:
    - predictions: Etiquetas predichas para los documentos de prueba.
    """
    from collections import Counter
    import numpy as np

    predictions = []
    for test_vec in document_vectors_test:
        # Calcular distancias a todos los documentos de entrenamiento
        distances = np.linalg.norm(document_vectors_train - test_vec, axis=1)
        # Obtener índices de los k vecinos más cercanos
        neighbor_indices = distances.argsort()[:k]
        # Obtener etiquetas de los vecinos más cercanos
        neighbor_labels = [labels_train[idx] for idx in neighbor_indices]
        # Predecir la etiqueta más común
        most_common = Counter(neighbor_labels).most_common(1)[0][0]
        predictions.append(most_common)

    return predictions


In [115]:
def evaluate_classification(true_labels, predicted_labels):
    """
    Evaluar la exactitud de la clasificación.

    Parámetros:
    - true_labels: Lista de etiquetas verdaderas.
    - predicted_labels: Lista de etiquetas predichas.

    Retorna:
    - accuracy: Exactitud de la clasificación.
    """
    correct = sum(t == p for t, p in zip(true_labels, predicted_labels))
    accuracy = correct / len(true_labels)
    return round(accuracy, 4)


In [116]:
import random

def split_data(documents, labels, test_size=0.2, random_state=42):
    """
    Divide los documentos y etiquetas en conjuntos de entrenamiento y prueba

    Args:
        documents (list): Lista de documentos tokenizados.
        labels (list): Lista de etiquetas correspondientes a cada documento.
        test_size (float): Proporción del conjunto de prueba.
        random_state (int): Semilla para el generador de números aleatorios.

    Returns:
        tuple: (documents_train, documents_test, labels_train, labels_test)
    """
    # Mezclar los datos para asegurar aleatoriedad
    data = list(zip(documents, labels))
    random.seed(random_state)
    random.shuffle(data)
    documents, labels = zip(*data)

    # Calcular el tamaño del conjunto de prueba
    split_index = int(len(documents) * (1 - test_size))
    documents_train, documents_test = documents[:split_index], documents[split_index:]
    labels_train, labels_test = labels[:split_index], labels[split_index:]

    return list(documents_train), list(documents_test), list(labels_train), list(labels_test)


In [117]:
import time
import tracemalloc
import numpy as np

# Establecer parámetros
k_dimensions = 80  # Dimensiones reducidas

# Implementación Inicial
start_time = time.time()
tracemalloc.start()

# LSA inicial
term_vectors_initial, doc_vectors_initial_train, term_index = lsa_initial(documents, k_dimensions)

current, peak = tracemalloc.get_traced_memory()
end_time = time.time()
tracemalloc.stop()

print(f"Tiempo de LSA inicial: {end_time - start_time} segundos")
print(f"Uso de memoria LSA inicial: Actual={current / 10**6} MB; Pico={peak / 10**6} MB")


Tiempo de LSA inicial: 185.9575126171112 segundos
Uso de memoria LSA inicial: Actual=14.98612 MB; Pico=1548.46632 MB


In [118]:
# Implementación Mejorada
start_time = time.time()
tracemalloc.start()

# LSA mejorada
term_vectors_improved, doc_vectors_improved_train, term_index = lsa_improved(documents, k_dimensions)

current, peak = tracemalloc.get_traced_memory()
end_time = time.time()
tracemalloc.stop()

print(f"Tiempo de LSA con mejoras: {end_time - start_time} segundos")
print(f"Uso de memoria LSA con mejoras: Actual={current / 10**6} MB; Pico={peak / 10**6} MB")

Tiempo de LSA con mejoras: 83.52752566337585 segundos
Uso de memoria LSA con mejoras: Actual=15.042239 MB; Pico=48.805111 MB


# Word2Ve

# Preprocesamiento

In [2]:
import urllib.request
import json

# Lista de títulos de artículos de Wikipedia en español
articulos = [
    "Inteligencia artificial", "Aprendizaje automático", "Biotecnología", "Nanotecnología", "Genética",
    "Tecnología de la información", "Robótica", "Revolución Industrial", "Segunda Guerra Mundial", "Edad Media",
    "Guerra Fría", "Revolución francesa", "Historia de la ciencia", "Renacimiento", "Literatura en español",
    "Pintura renacentista", "Música clásica"
]

# Base URL de la API de Wikipedia para obtener el extracto en formato JSON
base_url = 'https://es.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&explaintext&titles={}'

# Archivo para guardar el corpus
filename = 'corpus.txt'

with open(filename, 'w', encoding='utf-8') as file:
    for title in articulos:
        # Codificar el título para URL
        encoded_title = urllib.request.quote(title)
        url = base_url.format(encoded_title)
        #print(f"Descargando el artículo: {title}...")
        response = urllib.request.urlopen(url)
        data = response.read().decode('utf-8')
        # Extraer el texto del JSON
        json_data = json.loads(data)
        pages = json_data['query']['pages']
        page = next(iter(pages.values()))
        extract = page.get('extract', '')
        # Escribir el texto en el archivo si el extracto no está vacío
        if extract:
            file.write(extract + '\n')
            #print(f"Artículo '{title}' descargado.")
        else:
            print(f"No se pudo descargar el artículo '{title}'.")
print(f"Todos los artículos han sido descargados y guardados en '{filename}'.")

Todos los artículos han sido descargados y guardados en 'corpus.txt'.


In [3]:
import string
from collections import Counter

def tokenize(text):
    # Convertir todo a minúsculas
    text = text.lower()
    # Remover puntuación
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    # Dividir en palabras
    tokens = text.split()
    return tokens

def stem(word):
    suffixes = ['aciones', 'imientos', 'amiento', 'ición', 'adora', 'ación', 'adoras', 'adores', 'ante',
                'ancia', 'mente', 'idad', 'ivas', 'ivos', 'anza', 'icos', 'icas', 'ico', 'ica',
                'oso', 'osa', 'osos', 'osas', 'ismo', 'ismos', 'able', 'ables', 'ible', 'ibles',
                'ente', 'entes', 'mente']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

stopwords = set([
    'a', 'al', 'algo', 'algunas', 'algunos', 'ante', 'antes', 'como',
    'con', 'contra', 'de', 'del', 'desde', 'donde', 'durante', 'e',
    'el', 'ella', 'ellas', 'ellos', 'en', 'entre', 'era', 'erais',
    'eran', 'eras', 'eres', 'es', 'esa', 'esas', 'ese', 'eso',
    'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas',
    'estad', 'estada', 'estadas', 'estado', 'estados', 'estamos',
    'estando', 'estar', 'estaremos', 'estará', 'estarán', 'estarás',
    'estaré', 'estaréis', 'estaría', 'estaríais', 'estaríamos',
    'estarían', 'estarías', 'estas', 'este', 'estemos', 'esto',
    'estos', 'estoy', 'estuve', 'estuviera', 'estuvierais',
    'estuvieran', 'estuvieras', 'estuvieron', 'estuviese',
    'estuvieseis', 'estuviesen', 'estuvieses', 'estuvimos',
    'estuviste', 'estuvisteis', 'estuviéramos', 'estuviésemos',
    'estuvo', 'está', 'estábamos', 'estáis', 'están', 'estás',
    'esté', 'estéis', 'estén', 'estés', 'fue', 'fuera', 'fuerais',
    'fueran', 'fueras', 'fueron', 'fuese', 'fueseis', 'fuesen',
    'fueses', 'fui', 'fuimos', 'fuiste', 'fuisteis', 'ha', 'habida',
    'habidas', 'habido', 'habidos', 'habiendo', 'habremos', 'habrá',
    'habrán', 'habrás', 'habré', 'habréis', 'habría', 'habríais',
    'habríamos', 'habrían', 'habrías', 'habéis', 'había', 'habíais',
    'habíamos', 'habían', 'habías', 'han', 'has', 'hasta', 'hay',
    'haya', 'hayamos', 'hayan', 'hayas', 'hayáis', 'he', 'hemos',
    'hube', 'hubiera', 'hubierais', 'hubieran', 'hubieras',
    'hubieron', 'hubiese', 'hubieseis', 'hubiesen', 'hubieses',
    'hubimos', 'hubiste', 'hubisteis', 'hubiéramos', 'hubiésemos',
    'hubo', 'la', 'las', 'le', 'les', 'lo', 'los', 'me', 'mi',
    'mis', 'mucho', 'muchos', 'muy', 'más', 'mí', 'mía', 'mías',
    'mío', 'míos', 'nada', 'ni', 'no', 'nos', 'nosotras', 'nosotros',
    'nuestra', 'nuestras', 'nuestro', 'nuestros', 'o', 'os', 'otra',
    'otras', 'otro', 'otros', 'para', 'pero', 'poco', 'por', 'porque',
    'que', 'quien', 'quienes', 'qué', 'se', 'sea', 'seamos', 'sean',
    'seas', 'seremos', 'será', 'serán', 'serás', 'seré', 'seréis',
    'sería', 'seríais', 'seríamos', 'serían', 'serías', 'seáis',
    'si', 'sido', 'siendo', 'sin', 'sobre', 'sois', 'somos', 'son',
    'soy', 'su', 'sus', 'suya', 'suyas', 'suyo', 'suyos', 'sí', 'también',
    'tanto', 'te', 'tendremos', 'tendrá', 'tendrán', 'tendrás',
    'tendré', 'tendréis', 'tendría', 'tendríais', 'tendríamos',
    'tendrían', 'tendrías', 'tened', 'tenemos', 'tenga', 'tengamos',
    'tengan', 'tengas', 'tengo', 'tengáis', 'tenida', 'tenidas',
    'tenido', 'tenidos', 'teniendo', 'tenéis', 'tenía', 'teníais',
    'teníamos', 'tenían', 'tenías', 'ti', 'tiene', 'tienen', 'tienes',
    'todo', 'todos', 'tu', 'tus', 'tuve', 'tuviera', 'tuvierais',
    'tuvieran', 'tuvieras', 'tuvieron', 'tuviese', 'tuvieseis',
    'tuviesen', 'tuvieses', 'tuvimos', 'tuviste', 'tuvisteis',
    'tuviéramos', 'tuviésemos', 'tuvo', 'tú', 'un', 'una', 'uno',
    'unos', 'vosotras', 'vosotros', 'vuestra', 'vuestras', 'vuestro',
    'vuestros', 'y', 'ya', 'yo', 'él', 'éramos'
])

print("Iniciando el preprocesamiento del corpus...")
with open('corpus.txt', 'r', encoding='utf-8') as file:
    # Procesar el archivo línea por línea para manejar archivos grandes
    word_counter = Counter()
    final_tokens = []
    for line in file:
        tokens = tokenize(line)
        stems = [stem(token) for token in tokens]
        tokens_filtered = [word for word in stems if word not in stopwords]
        word_counter.update(tokens_filtered)
        final_tokens.extend(tokens_filtered)

# Establecer el umbral de frecuencia
threshold = 5

# Obtener palabras que cumplen el umbral
frequent_words = {word for word, count in word_counter.items() if count >= threshold}

# Filtrar las palabras raras
final_tokens = [word for word in final_tokens if word in frequent_words]

print(f"Tamaño del vocabulario final: {len(set(final_tokens))} palabras")
print("Preprocesamiento completado.")

Iniciando el preprocesamiento del corpus...
Tamaño del vocabulario final: 3281 palabras
Preprocesamiento completado.


## Implementación de CBOW

In [29]:
import numpy as np
import tracemalloc
import time

def build_vocabulary(tokens):
    """
    Construye el vocabulario a partir de los tokens y genera mapas de palabras a índices y viceversa.

    Args:
        tokens (list): Lista de tokens del corpus.

    Returns:
        tuple: (vocab_size, word_to_idx, idx_to_word)
            - vocab_size (int): Tamaño del vocabulario.
            - word_to_idx (dict): Diccionario que asigna a cada palabra un índice.
            - idx_to_word (dict): Diccionario que asigna a cada índice una palabra.
    """
    vocab = set(tokens)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    vocab_size = len(vocab)
    return vocab_size, word_to_idx, idx_to_word

def initialize_weights(vocab_size, embedding_dim):
    """
    Inicializa las matrices de pesos para los embeddings.

    Args:
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensión de los embeddings.

    Returns:
        tuple: (W1, W2)
            - W1 (np.ndarray): Matriz de pesos inicial entre input y capa oculta.
            - W2 (np.ndarray): Matriz de pesos inicial entre capa oculta y output.
    """
    W1 = np.random.uniform(-0.8, 0.8, (vocab_size, embedding_dim))
    W2 = np.random.uniform(-0.8, 0.8, (embedding_dim, vocab_size))
    return W1, W2

def generate_cbow_data(tokens, window_size):
    """
    Genera los datos de entrenamiento para el modelo CBOW.

    Args:
        tokens (list): Lista de tokens del corpus.
        window_size (int): Tamaño de la ventana de contexto.

    Returns:
        list: Lista de tuplas (context, target) para entrenamiento.
    """
    data = []
    for i in range(window_size, len(tokens) - window_size):
        context = tokens[i - window_size:i] + tokens[i + 1:i + window_size + 1]
        target = tokens[i]
        data.append((context, target))
    return data

def one_hot_encode(word_idx, vocab_size):
    """
    Codifica una palabra en un vector one-hot.

    Args:
        word_idx (int): Índice de la palabra a codificar.
        vocab_size (int): Tamaño del vocabulario.

    Returns:
        np.ndarray: Vector one-hot de longitud vocab_size con un 1 en la posición de word_idx.
    """
    vec = np.zeros(vocab_size)
    vec[word_idx] = 1
    return vec

def softmax(x):
    """
    Calcula la función softmax de un vector.

    Args:
        x (np.ndarray): Vector de entrada.

    Returns:
        np.ndarray: Vector con los valores softmax.
    """
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def train_cbow(training_data, W1, W2, word_to_idx, vocab_size, learning_rate, epochs, window_size):
    """
    Entrena el modelo CBOW utilizando descenso de gradiente y softmax.

    Args:
        training_data (list): Datos de entrenamiento (context, target).
        W1 (np.ndarray): Matriz de pesos entre input y capa oculta.
        W2 (np.ndarray): Matriz de pesos entre capa oculta y output.
        word_to_idx (dict): Diccionario que asigna índices a palabras.
        vocab_size (int): Tamaño del vocabulario.
        learning_rate (float): Tasa de aprendizaje para el descenso de gradiente.
        epochs (int): Número de épocas de entrenamiento.
        window_size (int): Tamaño de la ventana de contexto.

    Returns:
        tuple: (W1, W2) Matrices de pesos actualizadas.
    """
    for epoch in range(epochs):
        loss = 0
        for context_words, target_word in training_data:
            # Vector de entrada (promedio de los embeddings de las palabras de contexto)
            context_indices = [word_to_idx[word] for word in context_words]
            x = np.mean(W1[context_indices], axis=0)  # Shape: (embedding_dim,)

            # Cálculo hacia adelante
            z = np.dot(x, W2)  # Shape: (vocab_size,)
            y_pred = softmax(z)

            # Vector de salida deseado (one-hot encoding)
            y_true = one_hot_encode(word_to_idx[target_word], vocab_size)

            # Cálculo de la pérdida (entropía cruzada)
            loss -= np.log(y_pred[word_to_idx[target_word]] + 1e-9)

            # Cálculo del error
            e = y_pred - y_true  # Shape: (vocab_size,)

            # Cálculo hacia atrás (gradientes)
            dW2 = np.outer(x, e)  # Shape: (embedding_dim, vocab_size)
            dW1 = np.dot(W2, e) / (2 * window_size)  # Promedio para las palabras de contexto

            # Actualización de pesos
            W1[context_indices] -= learning_rate * dW1
            W2 -= learning_rate * dW2

        print(f"Epoch {epoch + 1}/{epochs}, Pérdida: {loss / len(training_data)}")
    return W1, W2

def measure_memory_time(func, *args, **kwargs):
    """
    Mide el tiempo de ejecución y el uso de memoria de una función.

    Args:
        func (callable): Función a medir.
        *args: Argumentos para la función.
        **kwargs: Argumentos nombrados para la función.

    Returns:
        tuple: (result, tiempo_ejecucion, current, peak)
            - result: Resultado de la función.
            - tiempo_ejecucion (float): Tiempo de ejecución en segundos.
            - current (float): Uso de memoria actual en MB.
            - peak (float): Uso máximo de memoria en MB.
    """
    tracemalloc.start()
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    tiempo_ejecucion = end_time - start_time
    return result, tiempo_ejecucion, current / 10**6, peak / 10**6

# Ejecución del modelo CBOW

# Definir los parámetros
embedding_dim = 50
window_size = 2
learning_rate = 0.05
epochs = 50

# Construir vocabulario
vocab_size, word_to_idx, idx_to_word = build_vocabulary(final_tokens)

# Inicializar pesos
W1, W2 = initialize_weights(vocab_size, embedding_dim)

# Generar datos de entrenamiento
training_data = generate_cbow_data(final_tokens, window_size)
print(f"Número de muestras de entrenamiento: {len(training_data)}")

# Entrenar y medir tiempo y memoria
(W1, W2), tiempo_ejecucion, current_mem, peak_mem = measure_memory_time(
    train_cbow, training_data, W1, W2, word_to_idx, vocab_size, learning_rate, epochs, window_size
)

# Resultados de tiempo y memoria
print(f"Tiempo de CBOW inicial: {tiempo_ejecucion} segundos")
print(f"Uso de memoria CBOW inicial: Actual={current_mem} MB; Pico={peak_mem} MB")

Número de muestras de entrenamiento: 55805
Epoch 1/50, Pérdida: 8.150338530613071
Epoch 2/50, Pérdida: 7.4808715852180425
Epoch 3/50, Pérdida: 6.980647591927219
Epoch 4/50, Pérdida: 6.5984203726249175
Epoch 5/50, Pérdida: 6.274850797763866
Epoch 6/50, Pérdida: 5.988151156826689
Epoch 7/50, Pérdida: 5.728892169815979
Epoch 8/50, Pérdida: 5.492144297698888
Epoch 9/50, Pérdida: 5.274788892160153
Epoch 10/50, Pérdida: 5.074720989628966
Epoch 11/50, Pérdida: 4.890112613342857
Epoch 12/50, Pérdida: 4.719334432518657
Epoch 13/50, Pérdida: 4.5610181967337295
Epoch 14/50, Pérdida: 4.413986118020979
Epoch 15/50, Pérdida: 4.277204574388703
Epoch 16/50, Pérdida: 4.149760320882509
Epoch 17/50, Pérdida: 4.0308404865246255
Epoch 18/50, Pérdida: 3.919710051012402
Epoch 19/50, Pérdida: 3.8156951331735347
Epoch 20/50, Pérdida: 3.718175700941419
Epoch 21/50, Pérdida: 3.626584087840843
Epoch 22/50, Pérdida: 3.5404057369027826
Epoch 23/50, Pérdida: 3.4591788343222056
Epoch 24/50, Pérdida: 3.382490459739585

# Implementación de skip-gram con negative sampling

In [30]:
import numpy as np
import time

def build_vocabulary(tokens):
    """
    Construye el vocabulario y mapea palabras a índices y viceversa.

    Args:
        tokens (list): Lista de tokens del corpus.

    Returns:
        tuple: (vocab_size, word_to_idx, idx_to_word)
            - vocab_size (int): Tamaño del vocabulario.
            - word_to_idx (dict): Diccionario que asigna índices a palabras.
            - idx_to_word (dict): Diccionario que asigna palabras a índices.
    """
    vocab = set(tokens)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    vocab_size = len(vocab)
    return vocab_size, word_to_idx, idx_to_word

def initialize_weights(vocab_size, embedding_dim):
    """
    Inicializa las matrices de pesos para las capas de embeddings.

    Args:
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensión de los embeddings.

    Returns:
        tuple: (W1, W2) Matrices de pesos inicializadas.
    """
    W1 = np.random.uniform(-0.8, 0.8, (vocab_size, embedding_dim))
    W2 = np.random.uniform(-0.8, 0.8, (embedding_dim, vocab_size))
    return W1, W2

def generate_skipgram_data(tokens, window_size):
    """
    Genera los datos de entrenamiento para el modelo Skip-Gram.

    Args:
        tokens (list): Lista de tokens del corpus.
        window_size (int): Tamaño de la ventana de contexto.

    Returns:
        list: Lista de tuplas (target, context) para entrenamiento.
    """
    data = []
    for i in range(len(tokens)):
        target = tokens[i]
        context_indices = list(range(max(0, i - window_size), i)) + list(range(i + 1, min(len(tokens), i + window_size + 1)))
        context = [tokens[idx] for idx in context_indices]
        for context_word in context:
            data.append((target, context_word))
    return data

def sigmoid(x):
    """
    Calcula la función sigmoide de un valor.

    Args:
        x (float or np.ndarray): Valor o array de entrada.

    Returns:
        float or np.ndarray: Valor o array con la función sigmoide aplicada.
    """
    return 1 / (1 + np.exp(-x))

def get_negative_samples(target_idx, num_neg_samples, vocab_size):
    """
    Genera muestras negativas aleatorias excluyendo la palabra objetivo.

    Args:
        target_idx (int): Índice de la palabra objetivo.
        num_neg_samples (int): Número de muestras negativas a generar.
        vocab_size (int): Tamaño del vocabulario.

    Returns:
        list: Lista de índices de las muestras negativas.
    """
    negative_samples = []
    while len(negative_samples) < num_neg_samples:
        neg_idx = np.random.randint(0, vocab_size)
        if neg_idx != target_idx:
            negative_samples.append(neg_idx)
    return negative_samples

def train_skipgram(training_data, word_to_idx, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples):
    """
    Entrena el modelo Skip-Gram con muestreo negativo.

    Args:
        training_data (list): Datos de entrenamiento (target, context).
        word_to_idx (dict): Diccionario que asigna índices a palabras.
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensión de los embeddings.
        learning_rate (float): Tasa de aprendizaje.
        epochs (int): Número de épocas de entrenamiento.
        num_neg_samples (int): Número de muestras negativas.

    Returns:
        tuple: (W1, W2) Matrices de pesos después del entrenamiento.
    """
    # Inicializar pesos
    W1 = np.random.randn(vocab_size, embedding_dim) * 0.01
    W2 = np.random.randn(embedding_dim, vocab_size) * 0.01

    for epoch in range(epochs):
        loss = 0
        for target_word, context_word in training_data:
            target_idx = word_to_idx[target_word]
            context_idx = word_to_idx[context_word]

            # Obtener muestras negativas
            negative_indices = get_negative_samples(target_idx, num_neg_samples, vocab_size)

            # Vectores para el target y el contexto
            w_target = W1[target_idx]
            w_context = W2[:, context_idx]

            # Producto punto positivo
            score = np.dot(w_target, w_context)
            loss_pos = -np.log(sigmoid(score) + 1e-9)
            grad_pos = sigmoid(score) - 1

            # Actualización de vectores para el par positivo
            W1[target_idx] -= learning_rate * grad_pos * w_context
            W2[:, context_idx] -= learning_rate * grad_pos * w_target
            loss += loss_pos

            # Actualización de vectores para muestras negativas
            for neg_idx in negative_indices:
                w_negative = W2[:, neg_idx]
                score_neg = np.dot(w_target, w_negative)
                loss_neg = -np.log(sigmoid(-score_neg) + 1e-9)
                grad_neg = (1 - sigmoid(-score_neg))

                W1[target_idx] -= learning_rate * grad_neg * w_negative
                W2[:, neg_idx] -= learning_rate * grad_neg * w_target
                loss += loss_neg

        print(f"Epoch {epoch + 1}/{epochs}, Pérdida: {loss / len(training_data)}")
    return W1, W2

def measure_training_time_memory(func, *args, **kwargs):
    """
    Mide el tiempo de ejecución y el uso de memoria de la función de entrenamiento.

    Args:
        func (callable): Función de entrenamiento a medir.
        *args: Argumentos para la función.
        **kwargs: Argumentos nombrados para la función.

    Returns:
        tuple: (result, tiempo_ejecucion, current_mem, peak_mem)
            - result: Resultado de la función de entrenamiento.
            - tiempo_ejecucion (float): Tiempo de ejecución en segundos.
            - current_mem (float): Uso de memoria actual en MB.
            - peak_mem (float): Uso máximo de memoria en MB.
    """
    tracemalloc.start()
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    tiempo_ejecucion = end_time - start_time
    return result, tiempo_ejecucion, current_mem / 10**6, peak_mem / 10**6

# Parámetros y Ejecución del modelo Skip-Gram

# Definir los parámetros
embedding_dim = 50
window_size = 2
learning_rate = 0.01
epochs = 50
num_neg_samples = 5

# Construir vocabulario
vocab_size, word_to_idx, idx_to_word = build_vocabulary(final_tokens)

# Generar datos de entrenamiento
training_data_sg = generate_skipgram_data(final_tokens, window_size)
print(f"Número de muestras de entrenamiento: {len(training_data_sg)}")

# Entrenar y medir tiempo de ejecución
(W1, W2), tiempo_ejecucion, current_mem, peak_mem = measure_training_time_memory(
    train_skipgram, training_data_sg, word_to_idx, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples
)


# Resultados del tiempo de ejecución
print(f"Tiempo de entrenamiento Skip-Gram inicial: {tiempo_ejecucion} segundos")
print(f"Uso de memoria CBOW con mejoras: Actual={current_mem} MB; Pico={peak_mem} MB")

Número de muestras de entrenamiento: 223230
Epoch 1/50, Pérdida: 4.15867491761536
Epoch 2/50, Pérdida: 4.125658239919439
Epoch 3/50, Pérdida: 3.4878207855883163
Epoch 4/50, Pérdida: 2.6787751195252234
Epoch 5/50, Pérdida: 2.4247308190645045
Epoch 6/50, Pérdida: 2.350016609365114
Epoch 7/50, Pérdida: 2.320956133911222
Epoch 8/50, Pérdida: 2.3017132610258826
Epoch 9/50, Pérdida: 2.28391191292416
Epoch 10/50, Pérdida: 2.2607671848563826
Epoch 11/50, Pérdida: 2.233405705380576
Epoch 12/50, Pérdida: 2.205683352797052
Epoch 13/50, Pérdida: 2.175820872680901
Epoch 14/50, Pérdida: 2.1457642948034277
Epoch 15/50, Pérdida: 2.117418745847363
Epoch 16/50, Pérdida: 2.0905372548731282
Epoch 17/50, Pérdida: 2.061761188686031
Epoch 18/50, Pérdida: 2.031669964148663
Epoch 19/50, Pérdida: 2.004781745223011
Epoch 20/50, Pérdida: 1.9757679620528352
Epoch 21/50, Pérdida: 1.9462197657310016
Epoch 22/50, Pérdida: 1.9182686195333132
Epoch 23/50, Pérdida: 1.89070036226386
Epoch 24/50, Pérdida: 1.86243776827865

# Implementación con mejoras de CBOW

In [ ]:
import numpy as np
from collections import Counter
from multiprocessing import Pool
import multiprocessing
import time
import tracemalloc

def build_vocabulary(tokens):
    """
    Construye el vocabulario y mapea palabras a índices y viceversa.

    Args:
        tokens (list): Lista de tokens del corpus.

    Returns:
        tuple: (vocab_size, word_to_idx, idx_to_word)
            - vocab_size (int): Tamaño del vocabulario.
            - word_to_idx (dict): Diccionario que asigna índices a palabras.
            - idx_to_word (dict): Diccionario que asigna palabras a índices.
    """
    vocab = set(tokens)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    vocab_size = len(vocab)
    return vocab_size, word_to_idx, idx_to_word

def initialize_weights(vocab_size, embedding_dim):
    """
    Inicializa las matrices de pesos utilizando inicialización Xavier/Glorot.

    Args:
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensión de los embeddings.

    Returns:
        tuple: (W_input, W_output) Matrices de pesos inicializadas.
            - W_input (np.ndarray): Matriz de pesos de entrada.
            - W_output (np.ndarray): Matriz de pesos de salida.
    """
    limit = np.sqrt(6 / (vocab_size + embedding_dim))
    W_input = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))
    W_output = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))
    return W_input, W_output

def split_data(data, num_chunks):
    """
    Divide los datos en partes para distribuir entre múltiples procesos.

    Args:
        data (list): Lista de datos de entrenamiento.
        num_chunks (int): Número de divisiones.

    Returns:
        list: Lista de partes divididas.
    """
    if num_chunks > len(data):
        num_chunks = len(data)
    return np.array_split(data, num_chunks)

def sigmoid(x):
    """
    Calcula la función sigmoide.

    Args:
        x (np.ndarray): Vector de entrada.

    Returns:
        np.ndarray: Vector con la función sigmoide aplicada.
    """
    return 1 / (1 + np.exp(-x))

def get_negative_samples(target_idx, num_neg_samples, vocab_size):
    """
    Genera muestras negativas aleatorias excluyendo la palabra objetivo.

    Args:
        target_idx (int): Índice de la palabra objetivo.
        num_neg_samples (int): Número de muestras negativas a generar.
        vocab_size (int): Tamaño del vocabulario.

    Returns:
        list: Lista de índices de las muestras negativas.
    """
    negative_samples = []
    while len(negative_samples) < num_neg_samples:
        neg_idx = np.random.randint(0, vocab_size)
        if neg_idx != target_idx and neg_idx not in negative_samples:
            negative_samples.append(neg_idx)
    return negative_samples

def train_cbow_process(args):
    """
    Entrena una porción de datos en el modelo CBOW utilizando aprendizaje paralelo.

    Args:
        args (tuple): Contiene los siguientes elementos:
            - data_chunk (list): Parte de los datos de entrenamiento.
            - word_to_idx (dict): Diccionario que asigna palabras a índices.
            - learning_rate (float): Tasa de aprendizaje.
            - num_neg_samples (int): Número de muestras negativas.
            - vocab_size (int): Tamaño del vocabulario.
            - embedding_dim (int): Dimensión de los embeddings.

    Returns:
        tuple: (total_loss, W_input, W_output)
            - total_loss (float): Pérdida total para la porción de datos.
            - W_input (np.ndarray): Gradientes acumulados de pesos de entrada.
            - W_output (np.ndarray): Gradientes acumulados de pesos de salida.
    """
    data_chunk, word_to_idx, learning_rate, num_neg_samples, vocab_size, embedding_dim = args
    W_input = np.zeros((vocab_size, embedding_dim))
    W_output = np.zeros((vocab_size, embedding_dim))
    total_loss = 0

    for context_words, target_word in data_chunk:
        context_indices = [word_to_idx[word] for word in context_words]
        target_idx = word_to_idx[target_word]

        v_input = np.mean(W_input[context_indices], axis=0)

        neg_indices = get_negative_samples(target_idx, num_neg_samples, vocab_size)

        indices = [target_idx] + neg_indices
        u_output = W_output[indices]

        scores = np.dot(u_output, v_input)
        labels = np.array([1] + [0] * len(neg_indices))

        sigmoid_scores = sigmoid(scores)

        loss = -np.sum(np.log(sigmoid_scores + 1e-9) * labels + np.log(1 - sigmoid_scores + 1e-9) * (1 - labels))
        total_loss += loss

        grad = sigmoid_scores - labels
        grad_W_output = np.outer(grad, v_input)
        grad_v_input = np.dot(grad, u_output)

        for idx in indices:
            W_output[idx] -= learning_rate * grad_W_output[indices.index(idx)]

        for idx in context_indices:
            W_input[idx] -= learning_rate * grad_v_input / len(context_indices)

    return total_loss, W_input, W_output

def train_cbow_parallel(training_data, word_to_idx, W_input, W_output, learning_rate, num_neg_samples, vocab_size, embedding_dim, num_processes, epochs):
    """
    Entrena el modelo CBOW en paralelo utilizando múltiples procesos.

    Args:
        training_data (list): Datos de entrenamiento.
        word_to_idx (dict): Diccionario que asigna palabras a índices.
        W_input (np.ndarray): Matriz de pesos de entrada.
        W_output (np.ndarray): Matriz de pesos de salida.
        learning_rate (float): Tasa de aprendizaje.
        num_neg_samples (int): Número de muestras negativas.
        vocab_size (int): Tamaño del vocabulario.
        embedding_dim (int): Dimensión de los embeddings.
        num_processes (int): Número de procesos paralelos.
        epochs (int): Número de épocas de entrenamiento.

    Returns:
        tuple: (W_input, W_output) Matrices de pesos entrenadas.
    """
    for epoch in range(epochs):
        np.random.shuffle(training_data)
        data_chunks = split_data(training_data, num_processes)
        args = [
            (
                chunk,
                word_to_idx,
                learning_rate,
                num_neg_samples,
                vocab_size,
                embedding_dim
            )
            for chunk in data_chunks
        ]

        with Pool(processes=num_processes) as pool:
            results = pool.map(train_cbow_process, args)

        total_loss = 0
        grad_W_input = np.zeros_like(W_input)
        grad_W_output = np.zeros_like(W_output)

        for loss_chunk, W_input_chunk, W_output_chunk in results:
            total_loss += loss_chunk
            grad_W_input += W_input_chunk
            grad_W_output += W_output_chunk

        W_input -= grad_W_input
        W_output -= grad_W_output

        avg_loss = total_loss / len(training_data)
        print(f"Epoch {epoch + 1}/{epochs}, Pérdida Promedio: {avg_loss}")

    return W_input, W_output

def measure_training_time_memory(func, *args, **kwargs):
    """
    Mide el tiempo de ejecución y el uso de memoria de la función de entrenamiento.

    Args:
        func (callable): Función de entrenamiento a medir.
        *args: Argumentos para la función.
        **kwargs: Argumentos nombrados para la función.

    Returns:
        tuple: (result, tiempo_ejecucion, current_mem, peak_mem)
            - result: Resultado de la función de entrenamiento.
            - tiempo_ejecucion (float): Tiempo de ejecución en segundos.
            - current_mem (float): Uso de memoria actual en MB.
            - peak_mem (float): Uso máximo de memoria en MB.
    """
    tracemalloc.start()
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    tiempo_ejecucion = end_time - start_time
    return result, tiempo_ejecucion, current_mem / 10**6, peak_mem / 10**6

def generate_cbow_data(tokens, window_size):
    """
    Genera datos para el modelo CBOW.

    Args:
        tokens (list): Lista de tokens del corpus.
        window_size (int): Tamaño de la ventana de contexto.

    Returns:
        list: Lista de tuplas (context, target) donde context es el contexto
        de palabras y target es la palabra objetivo.
    """
    data = []
    for idx in range(window_size, len(tokens) - window_size):
        context = tokens[idx - window_size:idx] + tokens[idx + 1:idx + window_size + 1]
        target = tokens[idx]
        data.append((context, target))
    return data

# Parámetros y Ejecución del modelo CBOW paralelo

embedding_dim = 50
window_size = 2
learning_rate = 0.01
epochs = 5
num_neg_samples = 5
num_processes = multiprocessing.cpu_count()

vocab_size, word_to_idx, idx_to_word = build_vocabulary(final_tokens)

W_input, W_output = initialize_weights(vocab_size, embedding_dim)

training_data = generate_cbow_data(final_tokens, window_size)

(W_input, W_output), tiempo_ejecucion, current_mem, peak_mem = measure_training_time_memory(
    train_cbow_parallel, training_data, word_to_idx, W_input, W_output, learning_rate, num_neg_samples, vocab_size, embedding_dim, num_processes, epochs
)

print(f"Tiempo de ejecución de CBOW con mejoras: {tiempo_ejecucion} segundos")
print(f"Uso de memoria CBOW con mejoras: Actual={current_mem} MB; Pico={peak_mem} MB")


# Implementación con mejoras de Skip-gram con negative sampling

In [ ]:
import numpy as np
from collections import Counter
from multiprocessing import Pool, cpu_count
import time
import tracemalloc

def build_vocabulary(tokens):
    vocab = set(tokens)
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    vocab_size = len(vocab)
    return vocab_size, word_to_idx, idx_to_word

def initialize_weights(vocab_size, embedding_dim):
    limit = np.sqrt(6 / (vocab_size + embedding_dim))
    W_input = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))
    W_output = np.random.uniform(-limit, limit, (vocab_size, embedding_dim))
    return W_input, W_output

def generate_skipgram_data(tokens, window_size):
    data = []
    for i in range(len(tokens)):
        target = tokens[i]
        context_indices = list(range(max(0, i - window_size), i)) + list(range(i + 1, min(len(tokens), i + window_size + 1)))
        context = [tokens[idx] for idx in context_indices]
        for context_word in context:
            data.append((target, context_word))
    return data

def create_alias_table(word_counts, vocab_size, idx_to_word):
    total_count = sum(word_counts.values())
    probs = np.array([word_counts[idx_to_word[i]] ** 0.75 for i in range(vocab_size)])
    probs /= np.sum(probs)

    scaled_probs = probs * vocab_size
    alias = np.zeros(vocab_size, dtype=np.int32)
    prob = np.zeros(vocab_size)

    small = [i for i, sp in enumerate(scaled_probs) if sp < 1.0]
    large = [i for i, sp in enumerate(scaled_probs) if sp >= 1.0]

    while small and large:
        s = small.pop()
        l = large.pop()
        alias[s] = l
        prob[s] = scaled_probs[s]
        scaled_probs[l] = scaled_probs[l] - (1.0 - scaled_probs[s])

        if scaled_probs[l] < 1.0:
            small.append(l)
        else:
            large.append(l)

    prob = np.clip(scaled_probs, 0, 1)
    return prob, alias

def alias_sample(prob_table, alias_table, num_samples):
    K = len(prob_table)
    kk = np.random.randint(0, K, size=num_samples)
    rr = np.random.uniform(0, 1, size=num_samples)
    return np.where(rr < prob_table[kk], kk, alias_table[kk])

def split_data(data, num_processes):
    chunk_size = len(data) // num_processes
    return [data[i * chunk_size:(i + 1) * chunk_size] for i in range(num_processes)]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def train_skipgram_process(args):
    """
    Entrena una porción de datos en el modelo Skip-Gram utilizando aprendizaje paralelo.

    Args:
        args (tuple): Tupla que contiene:
            - data_chunk (list): Parte de los datos de entrenamiento.
            - W_input (np.ndarray): Matriz de pesos de entrada.
            - W_output (np.ndarray): Matriz de pesos de salida.
            - word_to_idx (dict)
            - prob_table (np.ndarray)
            - alias_table (np.ndarray)
            - num_neg_samples (int)
            - learning_rate (float)
    Returns:
        tuple: (total_loss, W_input_update, W_output_update) Pérdida total y matrices de pesos actualizadas.
    """
    (data_chunk, W_input, W_output, word_to_idx, prob_table, alias_table, num_neg_samples, learning_rate) = args
    total_loss = 0

    # Inicializar actualizaciones locales
    W_input_update = np.zeros_like(W_input)
    W_output_update = np.zeros_like(W_output)

    for target_word, context_word in data_chunk:
        target_idx = word_to_idx[target_word]
        context_idx = word_to_idx[context_word]

        neg_indices = alias_sample(prob_table, alias_table, num_neg_samples)
        neg_indices = [idx for idx in neg_indices if idx != context_idx]

        indices = [context_idx] + neg_indices
        u_output = W_output[indices]
        v_input = W_input[target_idx]
        scores = np.dot(u_output, v_input)
        labels = np.array([1] + [0] * len(neg_indices))
        sigmoid_scores = sigmoid(scores)

        loss = -np.sum(np.log(sigmoid_scores + 1e-9) * labels + np.log(1 - sigmoid_scores + 1e-9) * (1 - labels))
        total_loss += loss

        grad = sigmoid_scores - labels
        grad_W_output = np.outer(grad, v_input)
        grad_v_input = np.dot(grad, u_output)

        # Acumular gradientes
        W_output_update[indices] -= learning_rate * grad_W_output
        W_input_update[target_idx] -= learning_rate * grad_v_input

    return total_loss, W_input_update, W_output_update

def train_skipgram_parallel(training_data_sg, W_input, W_output, word_to_idx, prob_table, alias_table,
                            num_neg_samples, learning_rate, num_processes, epochs):
    """
    Entrena el modelo Skip-Gram utilizando aprendizaje paralelo.

    Returns:
        W_input, W_output, tiempo_ejecucion, current_mem, peak_mem
    """
    # Medir tiempo y memoria
    tracemalloc.start()
    start_time = time.time()

    for epoch in range(epochs):
        # Mezclar datos
        np.random.shuffle(training_data_sg)
        # Dividir datos
        data_chunks = split_data(training_data_sg, num_processes)
        # Preparar argumentos para cada proceso
        args = [ (data_chunk, W_input, W_output, word_to_idx, prob_table, alias_table, num_neg_samples, learning_rate)
                 for data_chunk in data_chunks ]

        with Pool(processes=num_processes) as pool:
            results = pool.map(train_skipgram_process, args)

        # Combinar actualizaciones
        total_loss = 0
        W_input_update = np.zeros_like(W_input)
        W_output_update = np.zeros_like(W_output)

        for res in results:
            loss, W_input_grad, W_output_grad = res
            total_loss += loss
            W_input_update += W_input_grad
            W_output_update += W_output_grad

        # Actualizar pesos
        W_input += W_input_update
        W_output += W_output_update

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")

    end_time = time.time()
    current_mem, peak_mem = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    tiempo_ejecucion = end_time - start_time
    return W_input, W_output, tiempo_ejecucion, current_mem / 10**6, peak_mem / 10**6

# Parámetros y Ejecución del modelo Skip-Gram con aprendizaje paralelo
embedding_dim = 50
window_size = 2
learning_rate = 0.01
epochs = 10  # Reducido para fines de ejemplo
num_neg_samples = 5
num_processes = cpu_count()

# Construir vocabulario y datos
vocab_size, word_to_idx, idx_to_word = build_vocabulary(final_tokens)
W_input, W_output = initialize_weights(vocab_size, embedding_dim)
training_data_sg = generate_skipgram_data(final_tokens, window_size)

# Crear word_counts y tablas Alias
word_counts = Counter(final_tokens)
prob_table, alias_table = create_alias_table(word_counts, vocab_size, idx_to_word)

# Entrenar y medir tiempo y memoria
W_input, W_output, tiempo_ejecucion, current_mem, peak_mem = train_skipgram_parallel(
    training_data_sg, W_input, W_output, word_to_idx, prob_table, alias_table,
    num_neg_samples, learning_rate, num_processes, epochs
)

# Resultados de tiempo y memoria
print(f"Tiempo de ejecución de Skip-Gram con mejoras: {tiempo_ejecucion} segundos")
print(f"Uso de memoria Skip-Gram con mejoras: Actual={current_mem} MB; Pico={peak_mem} MB")


# GloVe

## Implementación inicial

In [3]:
import urllib.request
import json

articulos = [
    "Inteligencia artificial", "Aprendizaje automático", "Biotecnología", "Nanotecnología", "Genética",
    "Tecnología de la información", "Robótica", "Revolución Industrial", "Segunda Guerra Mundial", "Edad Media",
    "Guerra Fría", "Revolución francesa", "Historia de la ciencia", "Renacimiento", "Literatura en español",
    "Pintura renacentista", "Música clásica", "Filosofía griega", "Arquitectura moderna", "Sociología",
    "Antropología", "Psicología", "Parques nacionales de España", "Economía de mercado", "Globalización",
    "Política internacional", "Corrupción política", "Economía de América Latina", "Nutrición humana",
    "Genómica", "Epidemiología", "Neurociencia", "Psicología clínica", "Medicina tradicional", "Salud mental",
    "Juegos Olímpicos", "Arquitectura gótica", "Ingeniería estructural"
]

# URL de la API de Wikipedia para obtener el extracto en formato JSON
base_url = 'https://es.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&explaintext&titles={}'

# Archivo para guardar el corpus
filename = 'corpus.txt'

with open(filename, 'w', encoding='utf-8') as file:
    for title in articulos:
        """
        Descarga el contenido de artículos de Wikipedia y los guarda en un archivo de texto.

        Args:
            title (str): El título del artículo de Wikipedia a descargar.

        Funcionalidad:
            - Codifica el título del artículo para incluirlo en una URL de solicitud a la API de Wikipedia.
            - Descarga el contenido del artículo en formato JSON y extrae el campo 'extract', que contiene el texto del artículo.
            - Si el extracto no está vacío, lo escribe en el archivo especificado y confirma la descarga en consola.
            - Si el extracto está vacío, indica que el artículo no se pudo descargar.

        Output:
            Archivo 'corpus.txt' con el contenido de todos los artículos descargados.
        """

        # Codificar el título para URL
        encoded_title = urllib.request.quote(title)
        url = base_url.format(encoded_title)
        print(f"Descargando el artículo: {title}")
        response = urllib.request.urlopen(url)
        data = response.read().decode('utf-8')

        # Extraer el texto del JSON
        json_data = json.loads(data)
        pages = json_data['query']['pages']
        page = next(iter(pages.values()))
        extract = page.get('extract', '')

        # Escribir el texto en el archivo si el extracto no está vacío
        if extract:
            file.write(extract + '\n')
            print(f"Artículo '{title}' descargado.")
        else:
            print(f"No se pudo descargar el artículo '{title}'.")

print(f"Todos los artículos han sido descargados y guardados en '{filename}'.")

Descargando el artículo: Inteligencia artificial
Artículo 'Inteligencia artificial' descargado.
Descargando el artículo: Aprendizaje automático
Artículo 'Aprendizaje automático' descargado.
Descargando el artículo: Biotecnología
Artículo 'Biotecnología' descargado.
Descargando el artículo: Nanotecnología
Artículo 'Nanotecnología' descargado.
Descargando el artículo: Genética
Artículo 'Genética' descargado.
Descargando el artículo: Tecnología de la información
Artículo 'Tecnología de la información' descargado.
Descargando el artículo: Robótica
Artículo 'Robótica' descargado.
Descargando el artículo: Revolución Industrial
Artículo 'Revolución Industrial' descargado.
Descargando el artículo: Segunda Guerra Mundial
Artículo 'Segunda Guerra Mundial' descargado.
Descargando el artículo: Edad Media
Artículo 'Edad Media' descargado.
Descargando el artículo: Guerra Fría
Artículo 'Guerra Fría' descargado.
Descargando el artículo: Revolución francesa
Artículo 'Revolución francesa' descargado.
De

In [4]:
import string
from collections import Counter

def tokenize(text):
    """
    Convierte el texto en minúsculas, remueve la puntuación y lo tokeniza en palabras.

    Args:
        text (str): Texto a procesar.

    Returns:
        list: Lista de palabras (tokens) extraídas del texto.
    """
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = text.split()
    return tokens

def stem(word):
    """
    Aplica una reducción simple de sufijos comunes en español para el stemming.

    Args:
        word (str): Palabra a la que se le aplicará el stemming.

    Returns:
        str: La palabra sin sufijos comunes.
    """
    suffixes = ['osos', 'osas', 'ismo', 'ismos', 'able', 'ables', 'ible', 'ibles',
                'ente', 'entes', 'mente', 'aciones', 'imientos', 'amiento', 'ición', 'adora', 'ación', 'adoras', 'adores', 'ante',
                'ancia', 'mente', 'idad', 'ivas', 'ivos', 'anza', 'icos', 'icas', 'ico', 'ica',
                'oso', 'osa']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

stopwords = set([
     'vuestro', 'vuestros', 'y', 'ya', 'yo', 'él', 'éramos','a', 'al', 'algo', 'algunas', 'algunos',
     'ante', 'antes', 'como', 'con', 'contra', 'de', 'del', 'desde', 'donde', 'durante', 'e', 'el',
     'ella', 'ellas', 'ellos', 'en', 'entre', 'era', 'erais', 'eran', 'eras', 'eres', 'es',
     'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estabais', 'estaban', 'estabas',
    'vosotras', 'vosotros', 'vuestra', 'vuestras'
])


def preprocess_corpus(filename):
    """
    Lee un archivo de texto, realiza tokenización, stemming, remueve stopwords y filtra palabras raras.

    Args:
        filename (str): Ruta del archivo de texto a procesar.

    Returns:
        list: Lista de tokens procesados que cumplen con un umbral de frecuencia.
    """
    print("Iniciando el preprocesamiento del corpus")
    with open(filename, 'r', encoding='utf-8') as file:
        word_counter = Counter()
        final_tokens = []
        for line in file:
            tokens = tokenize(line)
            stems = [stem(token) for token in tokens]
            tokens_filtered = [word for word in stems if word not in stopwords]
            word_counter.update(tokens_filtered)
            final_tokens.extend(tokens_filtered)

    # Filtrado de palabras raras
    threshold = 5
    frequent_words = {word for word, count in word_counter.items() if count >= threshold}
    final_tokens = [word for word in final_tokens if word in frequent_words]

    print(f"Tamaño del vocabulario final: {len(set(final_tokens))} palabras")
    return final_tokens

final_tokens = preprocess_corpus('corpus.txt')

Iniciando el preprocesamiento del corpus
Tamaño del vocabulario final: 4888 palabras


In [5]:
import numpy as np
from collections import defaultdict

def build_vocab(tokens):
    """
    Construye el vocabulario a partir de una lista de tokens.

    Args:
        tokens (list): Lista de palabras tokenizadas del corpus.

    Returns:
        tuple: Un conjunto de palabras únicas (vocabulario), un diccionario que asigna un ID a cada palabra (word_to_id),
               y un diccionario inverso de IDs a palabras (id_to_word).
    """
    vocab = set(tokens)
    word_to_id = {word: idx for idx, word in enumerate(vocab)}
    id_to_word = {idx: word for word, idx in word_to_id.items()}
    return vocab, word_to_id, id_to_word

def build_cooccurrence_matrix(tokens, vocab, word_to_id, window_size=5):
    """
    Construye una matriz de co-ocurrencia de palabras utilizando una ventana de contexto.

    Args:
        tokens (list): Lista de palabras tokenizadas.
        vocab (set): Conjunto de palabras únicas en el corpus.
        word_to_id (dict): Diccionario que asigna un ID único a cada palabra del vocabulario.
        window_size (int): Tamaño de la ventana de contexto para contar co-ocurrencias.

    Returns:
        defaultdict: Matriz de co-ocurrencia en formato de diccionario, donde las claves son pares (word_id, context_id)
                     y los valores son las frecuencias ponderadas de co-ocurrencia.
    """
    cooccurrences = defaultdict(float)
    for i, word in enumerate(tokens):
        word_id = word_to_id[word]
        start = max(0, i - window_size)
        end = min(len(tokens), i + window_size + 1)
        for j in range(start, end):
            if i != j:
                context_word = tokens[j]
                context_id = word_to_id[context_word]
                distance = abs(i - j)
                weight = 1.0 / distance  # Ponderación por distancia inversa
                cooccurrences[(word_id, context_id)] += weight
    return cooccurrences

window_size = 5  # Tamaño de la ventana de contexto

# Construcción del vocabulario
vocab, word_to_id, id_to_word = build_vocab(final_tokens)

# Construcción de la matriz de co-ocurrencia
cooccurrence= build_cooccurrence_matrix(final_tokens, vocab, word_to_id, window_size)


In [6]:
import random
import numpy as np

def initialize_embeddings(vocab_size, embedding_dim=50):
    """
    Inicializa matrices de embeddings y vectores de sesgo para el modelo.

    Args:
        vocab_size (int): Número total de palabras en el vocabulario.
        embedding_dim (int, opcional): Dimensionalidad de los vectores de embeddings.

    Returns:
        tuple: Contiene las matrices de embeddings (W y W_tilde) y los vectores de sesgo (b y b_tilde).
               - W (np.array): Matriz de embeddings para las palabras del vocabulario.
               - W_tilde (np.array): Matriz de embeddings para las palabras en el contexto.
               - b (np.array): Vector de sesgo para las palabras del vocabulario.
               - b_tilde (np.array): Vector de sesgo para las palabras en el contexto.
    """
    W = np.random.uniform(-0.5, 0.5, (vocab_size, embedding_dim))
    W_tilde = np.random.uniform(-0.5, 0.5, (vocab_size, embedding_dim))
    b = np.zeros(vocab_size)
    b_tilde = np.zeros(vocab_size)
    return W, W_tilde, b, b_tilde


embedding_dim = 50  # Dimensión de los embeddings

# Inicialización de embeddings y sesgos
W, W_tilde, b, b_tilde = initialize_embeddings(len(vocab), embedding_dim)


In [10]:
import math

# Hiperparámetros
X_max = 100
alpha = 0.75
learning_rate = 0.05
epochs = 100 

In [7]:
def weighting_function(x_ij):
    """
    Calcula un peso basado en la frecuencia de co-ocurrencia entre palabras,
    aplicando una ponderación máxima para valores altos de frecuencia.

    Args:
        x_ij (float): Frecuencia de co-ocurrencia entre dos palabras.

    Returns:
        float: Peso ajustado en función de la frecuencia; es menor que 1 si
               x_ij es menor que X_max y 1 en caso contrario.
    """
    if x_ij < X_max:
        return (x_ij / X_max) ** alpha
    return 1.0


In [8]:
import time
import tracemalloc
import random
import numpy as np

def train_embeddings(cooccurrences, W, W_tilde, b, b_tilde, epochs, learning_rate):
    """
    Entrena los embeddings ajustando vectores de palabras y sesgos,
    utilizando una función de ponderación en función de la frecuencia de co-ocurrencia.

    Args:
        cooccurrences (dict): Diccionario de co-ocurrencias con pares de palabras como claves y frecuencias como valores.
        W (np.array): Matriz de embeddings de palabras.
        W_tilde (np.array): Matriz de embeddings de contexto.
        b (np.array): Vector de sesgos para las palabras.
        b_tilde (np.array): Vector de sesgos para el contexto.
        epochs (int): Número de iteraciones de entrenamiento.
        learning_rate (float): Tasa de aprendizaje para la actualización de los parámetros.
    """
    cooccurrence_items = list(cooccurrences.items())
    inicio = time.time()
    tracemalloc.start()

    for epoch in range(epochs):
        total_cost = 0
        random.shuffle(cooccurrence_items)

        for (i, j), x_ij in cooccurrence_items:
            # Calcula el peso f(x_ij)
            f_ij = weighting_function(x_ij)

            # Cálculo del producto punto y el costo para el par de palabras
            w_i = W[i]
            w_j = W_tilde[j]
            bi = b[i]
            bj = b_tilde[j]

            dot_product = np.dot(w_i, w_j)
            log_x_ij = math.log(x_ij)

            # Costo de error para cada par
            error = dot_product + bi + bj - log_x_ij
            cost = f_ij * np.power(error, 2)
            total_cost += 0.5 * cost

            # Calcular gradiente
            grad = f_ij * error
            # Actualización de parámetros
            W[i] -= learning_rate * grad * w_j
            W_tilde[j] -= learning_rate * grad * w_i
            b[i] -= learning_rate * grad
            b_tilde[j] -= learning_rate * grad

        average_cost = total_cost / len(cooccurrence_items)
        print(f"Epoch {epoch + 1}/{epochs}, Costo promedio: {average_cost}")

    fin = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    tiempo_ejecucion = fin - inicio
    uso_memoria = peak / 10**6
    print(f"Tiempo total de ejecución: {tiempo_ejecucion} segundos")
    print(f"Uso de memoria pico: {uso_memoria} MB")


In [11]:
train_embeddings(cooccurrence, W, W_tilde, b, b_tilde, epochs, learning_rate)

Epoch 1/100, Costo promedio: 0.01959478085628495
Epoch 2/100, Costo promedio: 0.015277750936697995
Epoch 3/100, Costo promedio: 0.013130331901944082
Epoch 4/100, Costo promedio: 0.01173686852196658
Epoch 5/100, Costo promedio: 0.010777448850583113
Epoch 6/100, Costo promedio: 0.010068601194500822
Epoch 7/100, Costo promedio: 0.009515412717139486
Epoch 8/100, Costo promedio: 0.009074825120611512
Epoch 9/100, Costo promedio: 0.008711276628052609
Epoch 10/100, Costo promedio: 0.008401825756057324
Epoch 11/100, Costo promedio: 0.008131627730565763
Epoch 12/100, Costo promedio: 0.007894824078514167
Epoch 13/100, Costo promedio: 0.00768236994657698
Epoch 14/100, Costo promedio: 0.007493112190585492
Epoch 15/100, Costo promedio: 0.007313902181939885
Epoch 16/100, Costo promedio: 0.007151933590402577
Epoch 17/100, Costo promedio: 0.0070050872181588635
Epoch 18/100, Costo promedio: 0.006865083594179574
Epoch 19/100, Costo promedio: 0.006736626340488856
Epoch 20/100, Costo promedio: 0.0066122593

## Tareas de evaluación

In [12]:
embeddings_glove_initial = W + W_tilde

In [13]:
analogias = [
    ('hombre', 'mujer', 'rey', 'reina')
]

In [14]:
missing_words = set()
for analogia in analogias:
    for word in analogia:
        if word not in word_to_id:
            missing_words.add(word)

if missing_words:
    print("Las siguientes palabras no están en el vocabulario:", missing_words)
else:
    print("Todas las palabras de las analogías están en el vocabulario.")


Todas las palabras de las analogías están en el vocabulario.


In [15]:
import numpy as np

def evaluate_analogies(analogies, embeddings, word_to_idx, idx_to_word, top_n=1):
    correct = 0
    total = len(analogies)

    for word_a, word_b, word_c, word_d in analogies:
        if word_a not in word_to_idx or word_b not in word_to_idx or word_c not in word_to_idx:
            continue  # Saltar si alguna palabra no está en el vocabulario

        vec_a = embeddings[word_to_idx[word_a]]
        vec_b = embeddings[word_to_idx[word_b]]
        vec_c = embeddings[word_to_idx[word_c]]

        # Calcular el vector de analogía
        analogy_vector = vec_b - vec_a + vec_c

        # Calcular similitudes
        similarities = embeddings @ analogy_vector
        best_indices = np.argsort(-similarities)

        # Excluir las palabras originales
        best_indices = [idx for idx in best_indices if idx not in [word_to_idx[word_a], word_to_idx[word_b], word_to_idx[word_c]]]

        predicted_word = idx_to_word[best_indices[0]]

        if predicted_word == word_d:
            correct += 1

    accuracy = correct / total
    print(f"Exactitud en analogías: {accuracy * 100:.2f}% ({correct}/{total})")
    return accuracy


In [16]:
evaluate_analogies(analogias, embeddings_glove_initial, word_to_id, id_to_word)

Exactitud en analogías: 0.00% (0/1)


0.0

El valor de la exactitud de 0 en las analogías puede deberse a la falta de robustez de los corpus así como al ajuste de hiperparámetros

# Implementación mejorada de GloVe

In [17]:
import numpy as np
from collections import defaultdict

def build_vocab(tokens):
    """
    Construye el vocabulario a partir de una lista de tokens.

    Args:
        tokens (list): Lista de palabras tokenizadas del corpus.

    Returns:
        tuple: Un conjunto de palabras únicas (vocabulario), un diccionario que asigna un ID a cada palabra (word_to_id),
               y un diccionario inverso de IDs a palabras (id_to_word).
    """
    vocab = set(tokens)
    word_to_id = {word: idx for idx, word in enumerate(vocab)}
    id_to_word = {idx: word for word, idx in word_to_id.items()}
    return vocab, word_to_id, id_to_word

def build_cooccurrence_matrix(tokens, vocab, word_to_id, window_size=5):
    """
    Construye una matriz de co-ocurrencia de palabras utilizando una ventana de contexto.

    Args:
        tokens (list): Lista de palabras tokenizadas.
        vocab (set): Conjunto de palabras únicas en el corpus.
        word_to_id (dict): Diccionario que asigna un ID único a cada palabra del vocabulario.
        window_size (int): Tamaño de la ventana de contexto para contar co-ocurrencias.

    Returns:
        defaultdict: Matriz de co-ocurrencia en formato de diccionario, donde las claves son pares (word_id, context_id)
                     y los valores son las frecuencias ponderadas de co-ocurrencia.
    """
    cooccurrences = defaultdict(float)
    for i, word in enumerate(tokens):
        word_id = word_to_id[word]
        start = max(0, i - window_size)
        end = min(len(tokens), i + window_size + 1)
        for j in range(start, end):
            if i != j:
                context_word = tokens[j]
                context_id = word_to_id[context_word]
                distance = abs(i - j)
                weight = 1.0 / distance  # Ponderación por distancia inversa
                cooccurrences[(word_id, context_id)] += weight
    return cooccurrences

window_size = 5  # Tamaño de la ventana de contexto

# Construcción del vocabulario
vocab, word_to_id, id_to_word = build_vocab(final_tokens)

# Construcción de la matriz de co-ocurrencia
cooccurrence= build_cooccurrence_matrix(final_tokens, vocab, word_to_id, window_size)

In [25]:
from scipy.sparse import coo_matrix
vocab_size = len(vocab)
# Obtener listas de índices y valores para construir la matriz dispersa
rows = []
cols = []
data = []

for (i, j), x_ij in cooccurrence.items():
    rows.append(i)
    cols.append(j)
    data.append(x_ij)

# Crear la matriz de co-ocurrencia dispersa
X = coo_matrix((data, (rows, cols)), shape=(vocab_size, vocab_size), dtype=np.float32)

In [26]:
def precompute_weights_and_logs(X, weighting_function):
    """
    Precomputa los valores ponderados y los logaritmos de las co-ocurrencias,
    optimizando el proceso de entrenamiento.

    Args:
        X (scipy.sparse.coo_matrix): Matriz de co-ocurrencias en formato disperso.
        weighting_function (function): Función para calcular el peso de cada co-ocurrencia.

    Returns:
        tuple: Dos arreglos de numpy, `f_X_data` y `log_X_data`, que contienen los valores
               ponderados y logarítmicos de las co-ocurrencias en `X`.
               - f_X_data (np.array): Peso f(X_{ij}) de cada co-ocurrencia.
               - log_X_data (np.array): Logaritmo log(X_{ij}) de cada co-ocurrencia.
    """
    f_X_data = np.zeros_like(X.data)
    log_X_data = np.zeros_like(X.data)

    for idx in range(len(X.data)):
        x_ij = X.data[idx]
        f_X_data[idx] = weighting_function(x_ij)
        log_X_data[idx] = np.log(x_ij)

    return f_X_data, log_X_data

# Ejecución de precomputación
f_X_data, log_X_data = precompute_weights_and_logs(X, weighting_function)

In [28]:
def initialize_vectors_and_biases(vocab_size, embedding_dim):
    """
    Inicializa las matrices de embeddings y los vectores de sesgos para el modelo,
    asignando valores aleatorios a los embeddings y ceros a los sesgos.

    Args:
        vocab_size (int): Número de palabras en el vocabulario.
        embedding_dim (int): Dimensionalidad de los vectores de embeddings.

    Returns:
        tuple: Matrices de embeddings y vectores de sesgos inicializados.
               - W (np.array): Matriz de embeddings para palabras, con valores aleatorios.
               - W_tilde (np.array): Matriz de embeddings para contexto, con valores aleatorios.
               - b (np.array): Vector de sesgo para palabras, inicializado en ceros.
               - b_tilde (np.array): Vector de sesgo para contexto, inicializado en ceros.
    """
    W = np.random.uniform(-0.5, 0.5, (vocab_size, embedding_dim)).astype(np.float32)
    W_tilde = np.random.uniform(-0.5, 0.5, (vocab_size, embedding_dim)).astype(np.float32)
    b = np.zeros(vocab_size, dtype=np.float32)
    b_tilde = np.zeros(vocab_size, dtype=np.float32)

    return W, W_tilde, b, b_tilde

# Ejecución de la inicialización
W, W_tilde, b, b_tilde = initialize_vectors_and_biases(vocab_size, embedding_dim)

In [29]:
def initialize_gradient_squares(vocab_size, embedding_dim):
    """
    Inicializa los acumuladores de cuadrados de gradiente para las matrices de embeddings
    y los vectores de sesgos, necesarios para optimización adaptativa.

    Args:
        vocab_size (int): Número de palabras en el vocabulario.
        embedding_dim (int): Dimensionalidad de los vectores de embeddings.

    Returns:
        tuple: Acumuladores de cuadrados de gradiente inicializados con unos.
               - gradsq_W (np.array): Acumulador para los gradientes de embeddings de palabras.
               - gradsq_W_tilde (np.array): Acumulador para los gradientes de embeddings de contexto.
               - gradsq_b (np.array): Acumulador para los gradientes de sesgo de palabras.
               - gradsq_b_tilde (np.array): Acumulador para los gradientes de sesgo de contexto.
    """
    gradsq_W = np.ones((vocab_size, embedding_dim), dtype=np.float32)
    gradsq_W_tilde = np.ones((vocab_size, embedding_dim), dtype=np.float32)
    gradsq_b = np.ones(vocab_size, dtype=np.float32)
    gradsq_b_tilde = np.ones(vocab_size, dtype=np.float32)

    return gradsq_W, gradsq_W_tilde, gradsq_b, gradsq_b_tilde

# Ejecución de la inicialización de acumuladores de gradiente
gradsq_W, gradsq_W_tilde, gradsq_b, gradsq_b_tilde = initialize_gradient_squares(vocab_size, embedding_dim)


In [30]:
import time
import numpy as np
import tracemalloc

def train_model_with_batches(X, f_X_data, log_X_data, W, W_tilde, b, b_tilde, gradsq_W, gradsq_W_tilde, gradsq_b, gradsq_b_tilde, epochs, learning_rate, batch_size=2048):
    """
    Entrena el modelo de embeddings en múltiples épocas, dividiendo los datos en lotes para optimizar la actualización
    de parámetros. Utiliza AdaGrad para una optimización adaptativa si los acumuladores de gradiente están definidos.

    Args:
        X (scipy.sparse.coo_matrix): Matriz dispersa de co-ocurrencias.
        f_X_data (np.array): Pesos precomputados para cada co-ocurrencia.
        log_X_data (np.array): Logaritmos precomputados de cada co-ocurrencia.
        W (np.array): Matriz de embeddings de palabras.
        W_tilde (np.array): Matriz de embeddings de contexto.
        b (np.array): Vector de sesgos para palabras.
        b_tilde (np.array): Vector de sesgos para contexto.
        gradsq_W (np.array): Acumulador de gradientes de `W` para AdaGrad.
        gradsq_W_tilde (np.array): Acumulador de gradientes de `W_tilde` para AdaGrad.
        gradsq_b (np.array): Acumulador de gradientes de `b` para AdaGrad.
        gradsq_b_tilde (np.array): Acumulador de gradientes de `b_tilde` para AdaGrad.
        epochs (int): Número total de épocas para el entrenamiento.
        learning_rate (float): Tasa de aprendizaje.
        batch_size (int, opcional): Tamaño de cada lote para la actualización.
    """
    inicio = time.time()
    tracemalloc.start()

    for epoch in range(epochs):
        total_cost = 0
        permutation = np.random.permutation(len(X.data))
        X_data_shuffled = X.data[permutation]
        f_X_data_shuffled = f_X_data[permutation]
        log_X_data_shuffled = log_X_data[permutation]
        row_indices = X.row[permutation]
        col_indices = X.col[permutation]

        num_batches = int(np.ceil(len(X_data_shuffled) / batch_size))

        for batch_idx in range(num_batches):
            start = batch_idx * batch_size
            end = min((batch_idx + 1) * batch_size, len(X_data_shuffled))

            i_s = row_indices[start:end]
            j_s = col_indices[start:end]
            x_ijs = X_data_shuffled[start:end]
            f_ijs = f_X_data_shuffled[start:end]
            log_x_ijs = log_X_data_shuffled[start:end]

            w_i = W[i_s]
            w_j = W_tilde[j_s]
            b_i = b[i_s]
            b_j = b_tilde[j_s]

            dot_products = np.sum(w_i * w_j, axis=1)
            errors = (dot_products + b_i + b_j - log_x_ijs)
            costs = f_ijs * errors ** 2
            total_cost += 0.5 * np.sum(costs)

            grad = f_ijs * errors

            grad_w_i = grad[:, np.newaxis] * w_j
            grad_w_j = grad[:, np.newaxis] * w_i

            if gradsq_W is not None:
                gradsq_W[i_s] += grad_w_i ** 2
                gradsq_W_tilde[j_s] += grad_w_j ** 2
                gradsq_b[i_s] += grad ** 2
                gradsq_b_tilde[j_s] += grad ** 2

                W[i_s] -= (learning_rate / np.sqrt(gradsq_W[i_s])) * grad_w_i
                W_tilde[j_s] -= (learning_rate / np.sqrt(gradsq_W_tilde[j_s])) * grad_w_j
                b[i_s] -= (learning_rate / np.sqrt(gradsq_b[i_s])) * grad
                b_tilde[j_s] -= (learning_rate / np.sqrt(gradsq_b_tilde[j_s])) * grad
            else:
                W[i_s] -= learning_rate * grad_w_i
                W_tilde[j_s] -= learning_rate * grad_w_j
                b[i_s] -= learning_rate * grad
                b_tilde[j_s] -= learning_rate * grad

        average_cost = total_cost / len(X.data)
        print(f"Epoch {epoch + 1}/{epochs}, Costo promedio: {average_cost}")

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    uso_memoria = peak / 10**6
    print(f"Tiempo total de ejecución: {tiempo_ejecucion} segundos")
    print(f"Uso de memoria pico: {uso_memoria} MB")


In [31]:
train_model_with_batches(X, f_X_data, log_X_data, W, W_tilde, b, b_tilde, gradsq_W, gradsq_W_tilde, gradsq_b, gradsq_b_tilde, epochs, learning_rate, batch_size=2048)

Epoch 1/100, Costo promedio: 0.02945303899735189
Epoch 2/100, Costo promedio: 0.02456833467961881
Epoch 3/100, Costo promedio: 0.02178615036928054
Epoch 4/100, Costo promedio: 0.019753966248337813
Epoch 5/100, Costo promedio: 0.018119247065061544
Epoch 6/100, Costo promedio: 0.016720278399914468
Epoch 7/100, Costo promedio: 0.015510732433338783
Epoch 8/100, Costo promedio: 0.014544588550946272
Epoch 9/100, Costo promedio: 0.013661633414290605
Epoch 10/100, Costo promedio: 0.012914699356060837
Epoch 11/100, Costo promedio: 0.012259993639432278
Epoch 12/100, Costo promedio: 0.011670634573155854
Epoch 13/100, Costo promedio: 0.01114636960560641
Epoch 14/100, Costo promedio: 0.01067463420399928
Epoch 15/100, Costo promedio: 0.01026386065948475
Epoch 16/100, Costo promedio: 0.009885646283670637
Epoch 17/100, Costo promedio: 0.009554518201733786
Epoch 18/100, Costo promedio: 0.009258546024394377
Epoch 19/100, Costo promedio: 0.008972534924792594
Epoch 20/100, Costo promedio: 0.00870670408735